In [2]:
import csv
import re
import pandas as pd
import collections
import numpy as np
from scipy import stats
from math import trunc
import threading
import multiprocessing as mp
from Bio import SeqIO
import pysam
import scipy
import math
import random
import seaborn as sns

## 1. 3nt periodicity

In [77]:
### Do the following for both Ribo-seq and RNA-seq data

sample = 'eIF1'
#rep = '1'
name = sample# + '-' + rep
SeqType = 'Ribo'
path = ('/Dell/Dell15/zhanggy/project/4-Ribo-seq-data/tmp/Ribo-seq_data_final/1-BAM/BAM/' + sample + '/') ## The pathway of bam file
path2 = ('/Dell/Dell15/zhanggy/project/4-Ribo-seq-data/tmp/Ribo-seq_data_final/1-BAM/BAM_gene/' + SeqType + '/' + sample + '/') ## The pathway to output results

# Put the information from the bam file into the csv file

# BAM_file = pysam.AlignmentFile((path + sample + '-' + SeqType + '-' + rep + '_Aligned.sortedByCoord.out.bam'), 'rb')
BAM_file = pysam.AlignmentFile((path + sample + '-' + SeqType + '_Aligned.sortedByCoord.out.bam'), 'rb')

BAM_csv = open((path2 + name + '_bam.csv'),'w') 
BAM_csv_writer = csv.writer(BAM_csv)
BAM_csv_writer.writerow(['gene','ReadsStartPos','cigar','seq'])

for line in BAM_file:
    BAM_csv_writer.writerow([line.reference_name, line.pos+1, line.cigarstring[0:-1], line.seq])
BAM_csv.close()

# Assign reads to individual genes

BAM_csv = pd.read_csv((path2 + name + '_bam.csv'))
BAM_csv.columns = ['seqid','ReadsStartPos','length','seq']
BAM_csv['length'] = BAM_csv['seq'].apply(len)
BAM_csv['ReadsEndPos'] = BAM_csv['ReadsStartPos'] + BAM_csv['length'] - 1

seqid = BAM_csv['seqid'].unique()
gene_position = pd.read_table('/Dell/Dell15/zhanggy/project/4-Ribo-seq-data/tmp/information/GeneMergeUTR_annotation.txt') ## Positional annotation of full-length gene transcripts, including UTRs
gene_position['start'] = gene_position['start'] - 20
gene_position['stop'] = gene_position['stop'] + 20

seqid_annotation = gene_position['chr'].unique()
seqid = [i for i in seqid if i in seqid_annotation]

ReadsAssignToGene = open((path2 + name + '_bam_AssignGene_transcript±20.csv'),'w')
ReadsAssignToGene_writer = csv.writer(ReadsAssignToGene)
ReadsAssignToGene_writer.writerow(['seqid','ReadsStartPos','ReadsEndPos','length','gene','strand','start','stop'])

for i in seqid:
    print(i)
    gene_position_subset = gene_position[gene_position['chr'] == i].reset_index()
    BAM_csv_subset = BAM_csv[BAM_csv['seqid'] == i].reset_index()
    for n in range(len(gene_position_subset['chr'])):
        start = gene_position_subset['start'][n]
        stop = gene_position_subset['stop'][n]
        gene = gene_position_subset['gene_id'][n]
        strand = gene_position_subset['strand'][n]
        BAM_csv_subset_pos = BAM_csv_subset[(BAM_csv_subset['ReadsStartPos'] >= start) & (BAM_csv_subset['ReadsEndPos'] <= stop)].reset_index()
        for m in range(len(BAM_csv_subset_pos['seqid'])):
            length = BAM_csv_subset_pos['length'][m]
            ReadsStartPos = BAM_csv_subset_pos['ReadsStartPos'][m]
            ReadsEndPos = BAM_csv_subset_pos['ReadsEndPos'][m]
            ReadsAssignToGene_writer.writerow([i,ReadsStartPos,ReadsEndPos,length,gene,strand,start,stop])
ReadsAssignToGene.close()

[E::idx_find_and_load] Could not retrieve index file for '/Dell/Dell15/zhanggy/project/4-Ribo-seq-data/tmp/Ribo-seq_data_final/1-BAM/BAM/eIF1/eIF1-Ribo_Aligned.sortedByCoord.out.bam'


I
II
III
IV
V
VI
VII
VIII
IX
X
XI
XII
XIII
XIV
XV
XVI
Mito


### a. All genes: 28nt reads in Ribo-seq and all length reads in RNA-seq


#### align all genes at start or stop codon

In [60]:
GenomeAnnotation = pd.read_table('/Dell/Dell15/zhanggy/project/4-Ribo-seq-data/tmp/information/Saccharomyces_cerevisiae.R64-1-1.107.gtf',skiprows=5,\
    names=['chr', 'DataBase', 'type','start','stop','6','strand','8','transcript_id'])
GenomeAnnotation = GenomeAnnotation[['chr','type','start','stop','strand','transcript_id']]
GenomeAnnotation_subset = GenomeAnnotation[(GenomeAnnotation['type'] == 'start_codon')|(GenomeAnnotation['type'] == 'stop_codon')]
GenomeAnnotation_subset = GenomeAnnotation_subset.reset_index().drop(['index'], axis = 1)

pattern = re.compile('"(.*)"')
for i in range(len(GenomeAnnotation_subset['transcript_id'])):
    GenomeAnnotation_subset['transcript_id'][i] = pattern.findall(GenomeAnnotation_subset['transcript_id'][i].split(';')[0])[0]

StartCodonPos = GenomeAnnotation_subset[GenomeAnnotation_subset['type'] == 'start_codon'].reset_index().drop(['index'], axis = 1)
StartCodonPos = StartCodonPos[['start','stop','strand','transcript_id']]
StartCodonPos.columns = ['StartCodonStart','StartCodonStop','strand','gene']
StopCodonPos = GenomeAnnotation_subset[GenomeAnnotation_subset['type'] == 'stop_codon'].reset_index().drop(['index'], axis = 1)
StopCodonPos = StopCodonPos[['start','stop','strand','transcript_id']]
StopCodonPos.columns = ['StopCodonStart','StopCodonStop','strand','gene']

/tmp/ipykernel_242461/2363197165.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  GenomeAnnotation_subset['transcript_id'][i] = pattern.findall(GenomeAnnotation_subset['transcript_id'][i].split(';')[0])[0]


In [95]:
### Do the following for both Ribo-seq and RNA-seq data

sample = 'HO'
#rep = '1'
name = sample# + '-' + rep
SeqType = 'RNA'
path1 = ('/Dell/Dell15/zhanggy/project/4-Ribo-seq-data/tmp/Ribo-seq_data_final/1-BAM/BAM_gene/' + SeqType + '/' + sample + '/') ## The pathway of bam_AssignGene_transcript±20.csv
path2 = ('/Dell/Dell15/zhanggy/project/4-Ribo-seq-data/tmp/Ribo-seq_data_final/2-TripletPeriodicity/Periodicity_AlignAtStartStopCodon/' + SeqType + '/' + sample + '/') ## The pathway to output results

# Judging the 5'/3' ends of reads relative to the start/stop codon

ReadsAssignToGene = pd.read_csv((path1 + name + '_bam_AssignGene_transcript±20.csv'))
ReadsAssignToGene = ReadsAssignToGene[(ReadsAssignToGene['gene'] != 'YJL189W')&(ReadsAssignToGene['gene'] != 'YLR075W')].reset_index(drop = True)
ReadsAssignToGene = pd.merge(ReadsAssignToGene,StartCodonPos,on = 'gene')
ReadsAssignToGene = pd.merge(ReadsAssignToGene,StopCodonPos,on = 'gene')
ReadsAssignToGene = ReadsAssignToGene[['seqid','ReadsStartPos','length','gene','StartCodonStart','StartCodonStop','StopCodonStart','StopCodonStop','strand']]

ReadsAssignToGene_1 = ReadsAssignToGene[ReadsAssignToGene['strand'] == '+'].reset_index().drop(['index'], axis = 1)
ReadsAssignToGene_1['StartPos_to_StartCodon'] = ReadsAssignToGene_1['ReadsStartPos'] - ReadsAssignToGene_1['StartCodonStart']
ReadsAssignToGene_1['StopPos_to_StartCodon'] = ReadsAssignToGene_1['StartPos_to_StartCodon'] + ReadsAssignToGene_1['length'] - 1
ReadsAssignToGene_1['StartPos_to_StopCodon'] = ReadsAssignToGene_1['ReadsStartPos'] - ReadsAssignToGene_1['StopCodonStart']
ReadsAssignToGene_1['StopPos_to_StopCodon'] = ReadsAssignToGene_1['StartPos_to_StopCodon'] + ReadsAssignToGene_1['length'] - 1
ReadsAssignToGene_1 = ReadsAssignToGene_1[['length','gene','strand','StartPos_to_StartCodon','StopPos_to_StartCodon','StartPos_to_StopCodon','StopPos_to_StopCodon']]

ReadsAssignToGene_0 = ReadsAssignToGene[ReadsAssignToGene['strand'] == '-'].reset_index().drop(['index'], axis = 1)
ReadsAssignToGene_0['ReadsEndPos'] = ReadsAssignToGene_0['ReadsStartPos'] + ReadsAssignToGene_0['length'] - 1
ReadsAssignToGene_0['StartPos_to_StartCodon'] = ReadsAssignToGene_0['StartCodonStop'] - ReadsAssignToGene_0['ReadsEndPos']
ReadsAssignToGene_0['StopPos_to_StartCodon'] = ReadsAssignToGene_0['StartCodonStop'] - ReadsAssignToGene_0['ReadsStartPos']
ReadsAssignToGene_0['StartPos_to_StopCodon'] = ReadsAssignToGene_0['StopCodonStop'] - ReadsAssignToGene_0['ReadsEndPos']
ReadsAssignToGene_0['StopPos_to_StopCodon'] = ReadsAssignToGene_0['StopCodonStop'] - ReadsAssignToGene_0['ReadsStartPos']
ReadsAssignToGene_0 = ReadsAssignToGene_0[['length','gene','strand','StartPos_to_StartCodon','StopPos_to_StartCodon','StartPos_to_StopCodon','StopPos_to_StopCodon']]

ReadsAssignToGene = pd.concat([ReadsAssignToGene_1,ReadsAssignToGene_0])
ReadsAssignToGene['frame'] = ReadsAssignToGene['StartPos_to_StartCodon'] % 3
ReadsAssignToGene.to_csv((path2 + name + '_ToStartAndStop_Frame.txt'),sep = '\t')


Length_Frame = ReadsAssignToGene.groupby('length')['frame'].value_counts()
Length_Frame.to_csv((path2 + name + '_Frame_length.txt'),sep = '\t')

Start_codon_around_count = ReadsAssignToGene.groupby('length')['StartPos_to_StartCodon'].value_counts().to_frame()
Start_codon_around_count.columns = ['counts']
Start_codon_around_count = Start_codon_around_count.reset_index()
Start_codon_around_count['frame'] = Start_codon_around_count['StartPos_to_StartCodon'] % 3
Start_codon_around_count.to_csv((path2 + name + '_StartCodonFrame_Length.txt'),sep = '\t')

Start_codon_total_count = ReadsAssignToGene['StartPos_to_StartCodon'].value_counts().to_frame()
Start_codon_total_count.columns = ['counts']
Start_codon_total_count = Start_codon_total_count.reset_index()
Start_codon_total_count.columns = ['StartPos_to_StartCodon','counts']
Start_codon_total_count['frame'] = Start_codon_total_count['StartPos_to_StartCodon'] % 3
Start_codon_total_count.to_csv((path2 + name + '_StartCodonFrame.txt'),sep = '\t')

Stop_codon_around_count = ReadsAssignToGene.groupby('length')['StartPos_to_StopCodon'].value_counts().to_frame()
Stop_codon_around_count.columns = ['counts']
Stop_codon_around_count = Stop_codon_around_count.reset_index()
Stop_codon_around_count['frame'] = Stop_codon_around_count['StartPos_to_StopCodon'] % 3
Stop_codon_around_count.to_csv((path2 + name + '_StopCodonFrame_Length.txt'),sep = '\t')

Stop_codon_total_count = ReadsAssignToGene['StartPos_to_StopCodon'].value_counts().to_frame()
Stop_codon_total_count.columns = ['counts']
Stop_codon_total_count = Stop_codon_total_count.reset_index()
Stop_codon_total_count.columns = ['StartPos_to_StopCodon','counts']
Stop_codon_total_count['frame'] = Stop_codon_total_count['StartPos_to_StopCodon'] % 3
Stop_codon_total_count.to_csv((path2 + name + '_StopCodonFrame.txt'),sep = '\t')


In [125]:
### combine RNA-seq and 28nt Ribo-seq 

sample = 'HO'
rep = '1'
name = sample + '-' + rep
path = '/Dell/Dell15/zhanggy/project/4-Ribo-seq-data/tmp/Ribo-seq_data_final/2-TripletPeriodicity/Periodicity_AlignAtStartStopCodon/'

Ribo_Start_codon = pd.read_table((path + 'Ribo/' + sample + '/' + name + '_StartCodonFrame_Length.txt'),usecols = ['length','StartPos_to_StartCodon','counts'])
Ribo_Start_codon = Ribo_Start_codon[Ribo_Start_codon['length'] == 28].reset_index(drop = True).drop(['length'], axis = 1)
Ribo_Start_codon.columns = ['StartPos_to_StartCodon','Ribo_counts']

RNA_Start_codon = pd.read_table((path + 'RNA/' + sample + '/' + name + '_StartCodonFrame.txt'), usecols = ['StartPos_to_StartCodon','counts'])
RNA_Start_codon.columns = ['StartPos_to_StartCodon','RNA_counts']

Ribo_RNA_Start_codon = pd.merge(Ribo_Start_codon,RNA_Start_codon,on = 'StartPos_to_StartCodon', how = 'outer')
Ribo_RNA_Start_codon['frame'] = Ribo_RNA_Start_codon['StartPos_to_StartCodon'] % 3
Ribo_RNA_Start_codon.to_csv((path + 'Ribo_RNA_Merge/' + sample + '/' + name + '_StartFrame.txt'), sep = '\t', na_rep = 0)

Ribo_Stop_codon = pd.read_table((path + 'Ribo/' + sample + '/' + name + '_StopCodonFrame_Length.txt'),usecols = ['length','StartPos_to_StopCodon','counts'])
Ribo_Stop_codon = Ribo_Stop_codon[Ribo_Stop_codon['length'] == 28].reset_index(drop = True).drop(['length'], axis = 1)
Ribo_Stop_codon.columns = ['StartPos_to_StopCodon','Ribo_counts']

RNA_Stop_codon = pd.read_table((path + 'RNA/' + sample + '/' + name + '_StopCodonFrame.txt'), usecols = ['StartPos_to_StopCodon','counts'])
RNA_Stop_codon.columns = ['StartPos_to_StopCodon','RNA_counts']

Ribo_RNA_Stop_codon = pd.merge(Ribo_Stop_codon,RNA_Stop_codon,on = 'StartPos_to_StopCodon', how = 'outer')
Ribo_RNA_Stop_codon['frame'] = Ribo_RNA_Stop_codon['StartPos_to_StopCodon'] % 3
Ribo_RNA_Stop_codon.to_csv((path + 'Ribo_RNA_Merge/' + sample + '/' + name + '_StopFrame.txt'), sep = '\t', na_rep = 0)

#### Frame 0/1/2 Fraction

In [78]:
sample = 'HO'
#rep = '1'
name = sample# + '-' + rep
SeqType = 'RNA'
path1 = ('/Dell/Dell15/zhanggy/project/4-Ribo-seq-data/tmp/Ribo-seq_data_final/2-TripletPeriodicity/Periodicity_AlignAtStartStopCodon/' + SeqType + '/' + sample + '/') ## _StartCodonFrame_Length.txt
path2 = ('/Dell/Dell15/zhanggy/project/4-Ribo-seq-data/tmp/Ribo-seq_data_final/2-TripletPeriodicity/Frame_Fraction/' + SeqType + '/' + sample + '/') ## The pathway to output results

StartCodon_Frame = pd.read_table((path1 + name + '_StartCodonFrame_Length.txt'))
#StartCodon_Frame = StartCodon_Frame[StartCodon_Frame['length'] == 28]
Frame_Fraction = StartCodon_Frame.groupby('frame')['counts'].sum().reset_index()
Frame_Fraction['fraction'] = Frame_Fraction['counts'] / Frame_Fraction['counts'].sum()
Frame_Fraction['sample'] = sample
#Frame_Fraction['rep'] = rep
Frame_Fraction.to_csv((path2 + name + '_Frame_Fraction.txt'), sep = '\t')
Frame_Fraction

,frame,counts,fraction,sample
0,0,2003541,0.344145,HO
1,1,1895669,0.325616,HO
2,2,1922575,0.330238,HO


In [83]:
## Ribo-seq and RNA-seq file merge

path = '/Dell/Dell15/zhanggy/project/4-Ribo-seq-data/tmp/Ribo-seq_data_final/2-TripletPeriodicity/Frame_Fraction/'

MutantList = ['HO-1','HO-2','CAF20-1','CAF20-2','EAP1-1','EAP1-2','eIF4E-1','eIF4E-2','eIF4G1-1','eIF4G1-2',\
    'PAB1-1','PAB1-2','eIF1-1','eIF1-2','eIF1A-1','eIF1A-2','eIF4A-1','eIF4A-2','eIF4B-1','eIF4B-2']         

for i in MutantList:
    locals()[('Ribo_Frame_' + i)] = pd.read_table((path + 'Ribo/' + i[:-2] + '/' + i + '_Frame_Fraction.txt'),usecols = ['frame','counts','fraction','sample','rep'])
    locals()[('Ribo_Frame_' + i)]['Type'] = 'Ribo_Seq'
    locals()[('RNA_Frame_' + i)] = pd.read_table((path + 'RNA/' + i[:-2] + '/' + i + '_Frame_Fraction.txt'),usecols = ['frame','counts','fraction','sample','rep'])
    locals()[('RNA_Frame_' + i)]['Type'] = 'RNA_Seq'


RiboAndRNA_Frame = pd.concat([locals()[('Ribo_Frame_HO-1')],locals()[('Ribo_Frame_HO-2')],locals()[('Ribo_Frame_CAF20-1')],locals()[('Ribo_Frame_CAF20-2')],\
    locals()[('Ribo_Frame_EAP1-1')],locals()[('Ribo_Frame_EAP1-2')],locals()[('Ribo_Frame_eIF4E-1')],locals()[('Ribo_Frame_eIF4E-2')],\
        locals()[('Ribo_Frame_eIF4G1-1')],locals()[('Ribo_Frame_eIF4G1-2')],locals()[('Ribo_Frame_PAB1-1')],locals()[('Ribo_Frame_PAB1-2')],\
            locals()[('Ribo_Frame_eIF1-1')],locals()[('Ribo_Frame_eIF1-2')],locals()[('Ribo_Frame_eIF1A-1')],locals()[('Ribo_Frame_eIF1A-2')],\
                locals()[('Ribo_Frame_eIF4A-1')],locals()[('Ribo_Frame_eIF4A-2')],locals()[('Ribo_Frame_eIF4B-1')],locals()[('Ribo_Frame_eIF4B-2')],\
    locals()[('RNA_Frame_HO-1')],locals()[('RNA_Frame_HO-2')],locals()[('RNA_Frame_CAF20-1')],locals()[('RNA_Frame_CAF20-2')],\
        locals()[('RNA_Frame_EAP1-1')],locals()[('RNA_Frame_EAP1-2')],locals()[('RNA_Frame_eIF4E-1')],locals()[('RNA_Frame_eIF4E-2')],\
            locals()[('RNA_Frame_eIF4G1-1')],locals()[('RNA_Frame_eIF4G1-2')],locals()[('RNA_Frame_PAB1-1')],locals()[('RNA_Frame_PAB1-2')],\
                locals()[('RNA_Frame_eIF1-1')],locals()[('RNA_Frame_eIF1-2')],locals()[('RNA_Frame_eIF1A-1')],locals()[('RNA_Frame_eIF1A-2')],\
                    locals()[('RNA_Frame_eIF4A-1')],locals()[('RNA_Frame_eIF4A-2')],locals()[('RNA_Frame_eIF4B-1')],locals()[('RNA_Frame_eIF4B-2')]]).reset_index(drop = 'True')
RiboAndRNA_Frame.to_csv((path +'AllSample_RegionFrame.txt'),sep = '\t')
RiboAndRNA_Frame

,frame,counts,fraction,sample,rep,Type
0,0,86431,0.853336,HO,1,Ribo_Seq
1,1,6533,0.064501,HO,1,Ribo_Seq
2,2,8322,0.082163,HO,1,Ribo_Seq
3,0,270502,0.849628,HO,2,Ribo_Seq
4,1,20595,0.064687,HO,2,Ribo_Seq
...,...,...,...,...,...,...
115,1,2530998,0.331780,eIF4B,1,RNA_Seq
116,2,2529417,0.331573,eIF4B,1,RNA_Seq
117,0,2694722,0.338078,eIF4B,2,RNA_Seq
118,1,2641394,0.331388,eIF4B,2,RNA_Seq


### b. Genes have uORF: 28nt reads in Ribo-seq

#### align genes at first uAUG

In [ ]:
### Finding the position of first uAUG at the 5'UTR


UTR3_annotation = pd.read_table('~/3UTR_seq.txt') ## Only consider genes with 5'UTR and 3'UTR annotations and 5'UTR length > 1
UTR3_gene = UTR3_annotation['gene_id'].unique()

UTR5_annotation = pd.read_table('~/5UTR_seq.txt')
UTR5_annotation['length'] = UTR5_annotation['seq'].apply(len)
UTR5_annotation = UTR5_annotation[UTR5_annotation['length'] != 1] 
UTR5_gene = UTR5_annotation['gene_id'].unique()

gene = list(set(UTR5_gene).intersection(set(UTR3_gene)))
UTR5_annotation = UTR5_annotation[UTR5_annotation['gene_id'].isin(gene)].reset_index(drop = True) 


First_uAUG = UTR5_annotation[UTR5_annotation['seq'].str.contains('ATG')]
uAUG_Gene = First_uAUG['gene_id'].unique()
print('n Gene have uAUG = ' + str(len(uAUG_Gene)))

First_uAUG_1 = First_uAUG[First_uAUG['strand'] == '+']
First_uAUG_1['position'] = First_uAUG_1['seq'].str.find('ATG')
First_uAUG_1['uAUG_start'] = First_uAUG_1['start'] + First_uAUG_1['position']
First_uAUG_1['uAUG_stop'] = First_uAUG_1['start'] + First_uAUG_1['position'] + 2
First_uAUG_1 = First_uAUG_1[['uAUG_start','uAUG_stop','strand','gene_id']]
First_uAUG_1.columns = ['uAUG_start','uAUG_stop','strand','gene']
First_uAUG_0 = First_uAUG[First_uAUG['strand'] == '-']
First_uAUG_0['position'] = First_uAUG_0['seq'].str.find('ATG')
First_uAUG_0['uAUG_start'] = First_uAUG_0['stop'] - First_uAUG_0['position'] - 2
First_uAUG_0['uAUG_stop'] = First_uAUG_0['stop'] - First_uAUG_0['position']
First_uAUG_0 = First_uAUG_0[['uAUG_start','uAUG_stop','strand','gene_id']]
First_uAUG_0.columns = ['uAUG_start','uAUG_stop','strand','gene']
First_uAUG = pd.concat([First_uAUG_0,First_uAUG_1]).reset_index(drop = True)

First_uAUG.to_csv('~/uAUG_Gene.txt', sep = '\t')

# Genes with uORFs annotated in the Ingolia article
Ingolia_uORF_GeneList = pd.read_table('~/uORF_Gene_2009-ingolia.txt')
Ingolia_uORF_Gene = Ingolia_uORF_GeneList['Gene'].unique()
First_uAUG = First_uAUG[First_uAUG['gene'].isin(Ingolia_uORF_Gene)].reset_index(drop = True)

First_uAUG.to_csv('~/uAUG_Gene_Ingolia_First.txt', sep = '\t')

In [155]:
sample = 'HO'
#rep = '1'
name = sample# + '-' + rep
path1 = ('/Dell/Dell15/zhanggy/project/4-Ribo-seq-data/tmp/Ribo-seq_data_final/1-BAM/BAM_gene/Ribo/' + sample + '/') ## The pathway of bam_AssignGene_transcript±20.csv
path2 = ('/Dell/Dell15/zhanggy/project/4-Ribo-seq-data/tmp/Ribo-seq_data_final/2-TripletPeriodicity/Periodicity_AlignAt_uAUG-2009-ingolia/First/' + sample + '/') ## The pathway to output results

First_uAUG = pd.read_table('/Dell/Dell15/zhanggy/project/4-Ribo-seq-data/tmp/Ribo-seq_data/5-InitiationFactor/1-TripletPeriodicity/uAUG_Gene_Ingolia_First.txt', usecols = ['uAUG_start','uAUG_stop','gene'])

ReadsAssignToGene = pd.read_csv((path1 + name + '_bam_AssignGene_transcript±20.csv'))
ReadsAssignToGene = pd.merge(ReadsAssignToGene, First_uAUG, on = 'gene')
ReadsAssignToGene = ReadsAssignToGene[['seqid','ReadsStartPos','ReadsEndPos','length','gene','uAUG_start','uAUG_stop','strand']]

ReadsAssignToGene_1 = ReadsAssignToGene[ReadsAssignToGene['strand'] == '+'].reset_index(drop = True)
ReadsAssignToGene_1['StartPos_to_uAUG_start'] = ReadsAssignToGene_1['ReadsStartPos'] - ReadsAssignToGene_1['uAUG_start']
ReadsAssignToGene_1['StopPos_to_uAUG_start'] = ReadsAssignToGene_1['ReadsEndPos'] - ReadsAssignToGene_1['uAUG_start']
ReadsAssignToGene_1 = ReadsAssignToGene_1[['length','gene','strand','StartPos_to_uAUG_start','StopPos_to_uAUG_start']]

ReadsAssignToGene_0 = ReadsAssignToGene[ReadsAssignToGene['strand'] == '-'].reset_index(drop = True)
ReadsAssignToGene_0['StartPos_to_uAUG_start'] = ReadsAssignToGene_0['uAUG_stop'] - ReadsAssignToGene_0['ReadsEndPos']
ReadsAssignToGene_0['StopPos_to_uAUG_start'] = ReadsAssignToGene_0['uAUG_stop'] - ReadsAssignToGene_0['ReadsStartPos']
ReadsAssignToGene_0 = ReadsAssignToGene_0[['length','gene','strand','StartPos_to_uAUG_start','StopPos_to_uAUG_start']]
ReadsAssignToGene = pd.concat([ReadsAssignToGene_1,ReadsAssignToGene_0]).reset_index(drop = True)
ReadsAssignToGene.to_csv((path2 + name + '_To_First_uAUG_Frame.txt'),sep = '\t')

LengthCount = ReadsAssignToGene['length'].value_counts().reset_index()
LengthCount.columns = ['length','count']
LengthCount.to_csv((path2 + name + '_Length_count.txt'),sep = '\t')

Start_codon_around_count = ReadsAssignToGene.groupby(['length'])['StartPos_to_uAUG_start'].value_counts().to_frame()
Start_codon_around_count.columns = ['counts']
Start_codon_around_count = Start_codon_around_count.reset_index()
Start_codon_around_count['frame'] = Start_codon_around_count['StartPos_to_uAUG_start'] % 3
Start_codon_around_count.to_csv((path2 + name + '_StartCodonFrame_Length.txt'),sep = '\t')

Gene_Start_codon_around_count = ReadsAssignToGene.groupby(['length','gene'])['StartPos_to_uAUG_start'].value_counts().to_frame() ## single gene level
Gene_Start_codon_around_count.columns = ['counts']
Gene_Start_codon_around_count = Gene_Start_codon_around_count.reset_index()
Gene_Start_codon_around_count['frame'] = Gene_Start_codon_around_count['StartPos_to_uAUG_start'] % 3
Gene_Start_codon_around_count.to_csv((path2 + name + '_Gene_StartCodonFrame_Length.txt'),sep = '\t')

Start_codon_total_count = ReadsAssignToGene['StartPos_to_uAUG_start'].value_counts().reset_index()
Start_codon_total_count.columns = ['StartPos_to_uAUG_start','counts']
Start_codon_total_count.to_csv((path2 + name + '_StartPos_count.txt'),sep = '\t')


#### align genes at second uAUG

In [ ]:
### Finding the position of second uAUG at the 5'UTR

UTR3_annotation = pd.read_table('~/3UTR_seq.txt') ## Only consider genes with 5'UTR and 3'UTR annotations and 5'UTR length > 1
UTR3_gene = UTR3_annotation['gene_id'].unique()

UTR5_annotation = pd.read_table('~/5UTR_seq.txt')
UTR5_annotation['length'] = UTR5_annotation['seq'].apply(len)
UTR5_annotation = UTR5_annotation[UTR5_annotation['length'] != 1] 
UTR5_gene = UTR5_annotation['gene_id'].unique()

gene = list(set(UTR5_gene).intersection(set(UTR3_gene)))
UTR5_annotation = UTR5_annotation[UTR5_annotation['gene_id'].isin(gene)].reset_index(drop = True) 


Second_uAUG = UTR5_annotation[UTR5_annotation['seq'].str.count('ATG') >= 2]
uAUG_Gene = Second_uAUG['gene_id'].unique()
print('n Gene have uAUG = ' + str(len(uAUG_Gene)))


def find_second_occurrence(text, substring):
    first_occurrence = text.find(substring)
    if first_occurrence != -1:
        second_occurrence = text.find(substring, first_occurrence + 1)
        return second_occurrence
    else:
        return -1

Second_uAUG_1 = Second_uAUG[Second_uAUG['strand'] == '+'].reset_index(drop = True)
Second_uAUG_1['position'] = ''
for i in range(len(Second_uAUG_1['position'])):
    Second_uAUG_1['position'][i] = find_second_occurrence(Second_uAUG_1['seq'][i],'ATG')
Second_uAUG_1['Second_uAUG_start'] = Second_uAUG_1['start'] + Second_uAUG_1['position']
Second_uAUG_1['Second_uAUG_stop'] = Second_uAUG_1['start'] + Second_uAUG_1['position'] + 2
Second_uAUG_1 = Second_uAUG_1[['Second_uAUG_start','Second_uAUG_stop','strand','gene_id']]
Second_uAUG_1.columns = ['Second_uAUG_start','Second_uAUG_stop','strand','gene']

Second_uAUG_0 = Second_uAUG[Second_uAUG['strand'] == '-'].reset_index(drop = True)
Second_uAUG_0['position'] = ''
for i in range(len(Second_uAUG_0['position'])):
    Second_uAUG_0['position'][i] = find_second_occurrence(Second_uAUG_0['seq'][i],'ATG')
Second_uAUG_0['Second_uAUG_start'] = Second_uAUG_0['stop'] - Second_uAUG_0['position'] - 2
Second_uAUG_0['Second_uAUG_stop'] = Second_uAUG_0['stop'] - Second_uAUG_0['position']
Second_uAUG_0 = Second_uAUG_0[['Second_uAUG_start','Second_uAUG_stop','strand','gene_id']]
Second_uAUG_0.columns = ['Second_uAUG_start','Second_uAUG_stop','strand','gene']
Second_uAUG = pd.concat([Second_uAUG_0,Second_uAUG_1]).reset_index(drop = True)

# Genes with uORFs annotated in the Ingolia article
Ingolia_uORF_GeneList = pd.read_table('~/uORF_Gene_2009-ingolia.txt')
Ingolia_uORF_Gene = Ingolia_uORF_GeneList['Gene'].unique()
Second_uAUG = Second_uAUG[Second_uAUG['gene'].isin(Ingolia_uORF_Gene)].reset_index(drop = True)

uAUG_First = pd.read_table('~/uAUG_Gene_Ingolia_First.txt', usecols = ['uAUG_start','uAUG_stop','gene'])
Second_uAUG = pd.merge(uAUG_First, Second_uAUG, on = 'gene')
Second_uAUG.to_csv('~/uAUG_Gene_Ingolia_Second.txt', sep = '\t')

In [185]:
sample = 'HO'
#rep = '2'
name = sample# + '-' + rep
path1 = ('/Dell/Dell15/zhanggy/project/4-Ribo-seq-data/tmp/Ribo-seq_data_final/1-BAM/BAM_gene/Ribo/' + sample + '/') ## The pathway of bam_AssignGene_transcript±20.csv
path2 = ('/Dell/Dell15/zhanggy/project/4-Ribo-seq-data/tmp/Ribo-seq_data_final/2-TripletPeriodicity/Periodicity_AlignAt_uAUG-2009-ingolia/Second/' + sample + '/') ## The pathway to output results


Second_uAUG = pd.read_table('/Dell/Dell15/zhanggy/project/4-Ribo-seq-data/tmp/Ribo-seq_data/5-InitiationFactor/1-TripletPeriodicity/uAUG_Gene_Ingolia_Second.txt', usecols = ['Second_uAUG_start','Second_uAUG_stop','gene'])

ReadsAssignToGene = pd.read_csv((path1 + name + '_bam_AssignGene_transcript±20.csv'))
ReadsAssignToGene = pd.merge(ReadsAssignToGene,Second_uAUG,on = 'gene')
ReadsAssignToGene = ReadsAssignToGene[['seqid','ReadsStartPos','ReadsEndPos','length','gene','Second_uAUG_start','Second_uAUG_stop','strand']]

ReadsAssignToGene_1 = ReadsAssignToGene[ReadsAssignToGene['strand'] == '+'].reset_index(drop = True)
ReadsAssignToGene_1['StartPos_to_Second_uAUG_start'] = ReadsAssignToGene_1['ReadsStartPos'] - ReadsAssignToGene_1['Second_uAUG_start']
ReadsAssignToGene_1['StopPos_to_Second_uAUG_start'] = ReadsAssignToGene_1['ReadsEndPos'] - ReadsAssignToGene_1['Second_uAUG_start']
ReadsAssignToGene_1 = ReadsAssignToGene_1[['length','gene','strand','StartPos_to_Second_uAUG_start','StopPos_to_Second_uAUG_start']]

ReadsAssignToGene_0 = ReadsAssignToGene[ReadsAssignToGene['strand'] == '-'].reset_index(drop = True)
ReadsAssignToGene_0['StartPos_to_Second_uAUG_start'] = ReadsAssignToGene_0['Second_uAUG_stop'] - ReadsAssignToGene_0['ReadsEndPos']
ReadsAssignToGene_0['StopPos_to_Second_uAUG_start'] = ReadsAssignToGene_0['Second_uAUG_stop'] - ReadsAssignToGene_0['ReadsStartPos']
ReadsAssignToGene_0 = ReadsAssignToGene_0[['length','gene','strand','StartPos_to_Second_uAUG_start','StopPos_to_Second_uAUG_start']]
ReadsAssignToGene = pd.concat([ReadsAssignToGene_1,ReadsAssignToGene_0]).reset_index(drop = True)
ReadsAssignToGene.to_csv((path2 + name + '_To_Second_uAUG_Frame.txt'),sep = '\t')

LengthCount = ReadsAssignToGene['length'].value_counts().reset_index()
LengthCount.columns = ['length','count']
LengthCount.to_csv((path2 + name + '_Length_count.txt'),sep = '\t')

Start_codon_around_count = ReadsAssignToGene.groupby(['length'])['StartPos_to_Second_uAUG_start'].value_counts().to_frame()
Start_codon_around_count.columns = ['counts']
Start_codon_around_count = Start_codon_around_count.reset_index()
Start_codon_around_count['frame'] = Start_codon_around_count['StartPos_to_Second_uAUG_start'] % 3
Start_codon_around_count.to_csv((path2 + name + '_StartCodonFrame_Length.txt'),sep = '\t')

Gene_Start_codon_around_count = ReadsAssignToGene.groupby(['length','gene'])['StartPos_to_Second_uAUG_start'].value_counts().to_frame()  ## single gene level
Gene_Start_codon_around_count.columns = ['counts']
Gene_Start_codon_around_count = Gene_Start_codon_around_count.reset_index()
Gene_Start_codon_around_count['frame'] = Gene_Start_codon_around_count['StartPos_to_Second_uAUG_start'] % 3
Gene_Start_codon_around_count.to_csv((path2 + name + '_Gene_StartCodonFrame_Length.txt'),sep = '\t')

Start_codon_total_count = ReadsAssignToGene['StartPos_to_Second_uAUG_start'].value_counts().reset_index()
Start_codon_total_count.columns = ['StartPos_to_Second_uAUG_start','counts']
Start_codon_total_count.to_csv((path2 + name + '_StartPos_count.txt'),sep = '\t')

## 2. Calculation of RPF abundance and RNA abundance

### a. RPF abundance and RNA abundance

In [217]:
### When assigning reads to genes, only the region with CDS±50 is selected here
### Do the following for both Ribo-seq and RNA-seq data

sample = 'EAP1'
rep = '1'
name = sample + '-' + rep
SeqType = 'RNA'
path2 = ('/Dell/Dell15/zhanggy/project/4-Ribo-seq-data/tmp/Ribo-seq_data_final/1-BAM/BAM_gene/' + SeqType + '/' + sample + '/') ## The pathway of bam.csv file

BAM_csv = pd.read_csv((path2 + name + '_bam.csv'))
BAM_csv.columns = ['seqid','start_pos','length','seq']

BAM_csv['length'] = BAM_csv['seq'].apply(len)
BAM_csv['stop_pos'] = BAM_csv['start_pos'] + BAM_csv['length'] - 1


seqid = BAM_csv['seqid'].unique()
gene_position = pd.read_table('/Dell/Dell15/zhanggy/project/4-Ribo-seq-data/tmp/information/Saccharomyces_cerevisiae_GeneInformation.txt') ## Positional annotation of gene CDS
gene_position['start'] = gene_position['start'] - 50
gene_position['stop'] = gene_position['stop'] + 50

seqid_annotation = gene_position['chr'].unique()
seqid = [i for i in seqid if i in seqid_annotation]

ReadsAssignToGene = open((path2 + name + '_bam_AssignGene_CDS±50.csv'),'w')
ReadsAssignToGene_writer = csv.writer(ReadsAssignToGene)
ReadsAssignToGene_writer.writerow(['seqid','ReadsStartPos','ReadsEndPos','length','gene','strand','start','stop'])
for i in seqid:
    print(i)
    gene_position_subset = gene_position[gene_position['chr'] == i].reset_index()
    BAM_csv_subset = BAM_csv[BAM_csv['seqid'] == i].reset_index()
    for n in range(len(gene_position_subset['chr'])):
        start = gene_position_subset['start'][n]
        stop = gene_position_subset['stop'][n]
        gene = gene_position_subset['gene_id'][n]
        strand = gene_position_subset['strand'][n]
        BAM_csv_subset_pos = BAM_csv_subset[(BAM_csv_subset['start_pos'] >= start) & (BAM_csv_subset['stop_pos'] <= stop)].reset_index()
        for m in range(len(BAM_csv_subset_pos['seqid'])):
            length = BAM_csv_subset_pos['length'][m]
            start_pos = BAM_csv_subset_pos['start_pos'][m]
            stop_pos = BAM_csv_subset_pos['stop_pos'][m]
            ReadsAssignToGene_writer.writerow([i,start_pos,stop_pos,length,gene,strand,start,stop])
ReadsAssignToGene.close()


I
II
III
IV
V
VI
VII
VIII
IX
X
XI
XII
XIII
XIV
XV
XVI
Mito


In [280]:
sample = 'EAP1'
rep = '2'
name = sample + '-' + rep
SeqType = 'Ribo'
path1 = ('/Dell/Dell15/zhanggy/project/4-Ribo-seq-data/tmp/Ribo-seq_data_final/1-BAM/BAM_gene/' + SeqType + '/' + sample + '/') ## The pathway of bam_AssignGene_CDS±50.csv file
path2 = ('/Dell/Dell15/zhanggy/project/4-Ribo-seq-data/tmp/Ribo-seq_data_final/3-RibosomeDensity/' + SeqType + '_Abundance/' + sample + '/') ## The pathway to output results

### Calculate all gene abundance

GeneReads = pd.read_csv((path1 + name + '_bam_AssignGene_CDS±50.csv'))
GeneReads.value_counts('gene').to_csv((path2 + name + '_ReadCount.txt'), sep = '\t')

GeneReads = pd.read_table((path2 + name + '_ReadCount.txt'))

gene_position = pd.read_table('/Dell/Dell15/zhanggy/project/4-Ribo-seq-data/tmp/information/Saccharomyces_cerevisiae_GeneInformation.txt') ## Positional annotation of gene CDS
gene_position = gene_position[['gene_id','start','stop']]
gene_position['length'] = gene_position['stop'] - gene_position['start'] + 1

GeneReads = pd.merge(GeneReads, gene_position, left_on = 'gene', right_on = 'gene_id')
GeneReads = GeneReads[['gene','0','length']]
GeneReads.columns = ['gene','count','length']
GeneReads_sum = GeneReads['count'].sum()
GeneReads[(SeqType + '_abundance')] = (GeneReads['count'] / GeneReads['length']) / GeneReads_sum
GeneReads['sample'] = name
GeneReads.to_csv((path2 + name + '-' + SeqType + '-Abundance.txt'), sep = '\t')

In [16]:
### Knock down and Knock out efficiency

path = '/Dell/Dell15/zhanggy/project/4-Ribo-seq-data/tmp/Ribo-seq_data_final/3-RibosomeDensity/RNA_Abundance/'
Gene_dict = {'ACT1':'YFL039C','EAP1':'YKL204W','CAF20':'YOR276W','eIF1':'YNL244C','eIF1A':'YMR260C',\
             'eIF4A':'YKR059W','eIF4B':'YPR163C','eIF4E':'YOL139C','eIF4G1':'YGR162W','PAB1':'YER165W'}
Gene_List = ['YFL039C','YKL204W','YOR276W','YNL244C','YMR260C','YKR059W','YPR163C','YOL139C','YGR162W','YER165W']

HO_1 = pd.read_table((path + 'HO/HO-1-RNA-Abundance.txt'),usecols = ['gene','RNA_abundance'])
HO_1['rep'] = 'rep1'
HO_1['RNA_abundance'] = HO_1['RNA_abundance'] / HO_1[HO_1['gene'] == 'YFL039C'].reset_index(drop = True)['RNA_abundance'][0]
HO_2 = pd.read_table((path + 'HO/HO-2-RNA-Abundance.txt'),usecols = ['gene','RNA_abundance'])
HO_2['rep'] = 'rep2'
HO_2['RNA_abundance'] = HO_2['RNA_abundance'] / HO_2[HO_2['gene'] == 'YFL039C'].reset_index(drop = True)['RNA_abundance'][0]
HO = pd.concat([HO_1,HO_2]).reset_index(drop = True)
HO['sample'] = 'HO'
HO = HO[HO['gene'].isin(Gene_List)].reset_index(drop = True)

sample = 'PAB1'
gene = Gene_dict[sample]
print(gene)

Mutant_1 = pd.read_table((path + sample + '/'  + sample + '-1-RNA-Abundance.txt'), usecols = ['gene','RNA_abundance'])
Mutant_1['rep'] = 'rep1'
Mutant_1['RNA_abundance'] = Mutant_1['RNA_abundance'] / Mutant_1[Mutant_1['gene'] == 'YFL039C'].reset_index(drop = True)['RNA_abundance'][0]

if gene not in Mutant_1['gene'].values: # 判断列中是否存在目标元素
    new_row = {'gene':gene,'RNA_abundance':0,'rep':'rep1'} # 如果目标元素不存在，则添加一行填充数据
    Mutant_1 = Mutant_1.append(new_row, ignore_index=True)

Mutant_2 = pd.read_table((path + sample + '/'  + sample + '-2-RNA-Abundance.txt'), usecols = ['gene','RNA_abundance'])
Mutant_2['rep'] = 'rep2'
Mutant_2['RNA_abundance'] = Mutant_2['RNA_abundance'] / Mutant_2[Mutant_2['gene'] == 'YFL039C'].reset_index(drop = True)['RNA_abundance'][0]

if gene not in Mutant_2['gene'].values:
    new_row = {'gene':gene,'RNA_abundance':0,'rep':'rep2'}
    Mutant_2 = Mutant_2.append(new_row, ignore_index=True)

Mutant = pd.concat([Mutant_1,Mutant_2]).reset_index(drop = True)
Mutant['sample'] = sample
Mutant = Mutant[Mutant['gene'].isin(Gene_List)].reset_index(drop = True)

HO_Mutant = pd.concat([HO,Mutant]).reset_index(drop = True)
HO_Mutant.to_csv((path + sample + '/' + sample + '_RNA_abundance_ACT1normalized.txt'), sep = '\t')
HO_Mutant[HO_Mutant['gene'] == gene].reset_index(drop = True).to_csv((path + sample + '/' + sample + '_RNA_abundance_ACT1normalized_subset.txt'), sep = '\t')
HO_Mutant[HO_Mutant['gene'] == gene].reset_index(drop = True)

YER165W


,gene,RNA_abundance,rep,sample
0,YER165W,0.858013,rep1,HO
1,YER165W,0.700748,rep2,HO
2,YER165W,0.131107,rep1,PAB1
3,YER165W,0.114185,rep2,PAB1


In [26]:
path = '/Dell/Dell15/zhanggy/project/4-Ribo-seq-data/tmp/Ribo-seq_data_final/3-RibosomeDensity/RNA_Abundance/'

MutantList = ['CAF20','EAP1','eIF4E','eIF4G1','PAB1','eIF1','eIF1A','eIF4A','eIF4B']             

for i in MutantList:
    locals()[('mRNA_' + i)] = pd.read_table((path + i + '/' + i + '_RNA_abundance_ACT1normalized_subset.txt'))
    locals()[('mRNA_' + i)]['group'] = i

AllSample = pd.concat([locals()[('mRNA_CAF20')],locals()[('mRNA_EAP1')],locals()[('mRNA_eIF4E')],locals()[('mRNA_eIF4G1')],locals()[('mRNA_PAB1')],
                       locals()[('mRNA_eIF1')],locals()[('mRNA_eIF1A')],locals()[('mRNA_eIF4A')],locals()[('mRNA_eIF4B')]]).reset_index().drop(['index','Unnamed: 0'], axis = 1)
AllSample.to_csv((path + 'AllSample_mRNA_ACT1normalized_subset.txt'), sep = '\t')
AllSample

,gene,RNA_abundance,rep,sample,group
0,YOR276W,0.085511,rep1,HO,CAF20
1,YOR276W,0.094737,rep2,HO,CAF20
2,YOR276W,0.001083,rep1,CAF20,CAF20
3,YOR276W,0.000298,rep2,CAF20,CAF20
4,YKL204W,0.029471,rep1,HO,EAP1
5,YKL204W,0.029167,rep2,HO,EAP1
6,YKL204W,0.001967,rep1,EAP1,EAP1
7,YKL204W,0.001484,rep2,EAP1,EAP1
8,YOL139C,0.270439,rep1,HO,eIF4E
9,YOL139C,0.287945,rep2,HO,eIF4E


In [7]:
path = '/Dell/Dell15/zhanggy/project/4-Ribo-seq-data/tmp/Ribo-seq_data_final/3-RibosomeDensity/'

#MutantList = ['HO-1','HO-2','CAF20-1','CAF20-2','EAP1-1','EAP1-2','eIF4E-1','eIF4E-2',\
#    'eIF4G1-1','eIF4G1-2','PAB1-1','PAB1-2','eIF1-1','eIF1-2','eIF1A-1','eIF1A-2','eIF4A-1','eIF4A-2','eIF4B-1','eIF4B-2']             

MutantList = ['HO','CAF20','EAP1','eIF4E','eIF4G1','PAB1','eIF1','eIF1A','eIF4A','eIF4B']   

for i in MutantList:
    #print(i)
    #locals()[('mut_f_' + i)] = pd.read_table((path + i[:-2] + '/' + i + '-Ribo-Abundance.txt'), usecols = ['gene','RNA_Abundance'])
    locals()[('mut_f_' + i)] = pd.read_table((path + 'Ribosome_Density/' + i + '/' + i + '-RibosomeDensity.txt'), usecols = ['gene','Ribosome_Density'])
    locals()[('mut_f_' + i)].columns = ['gene', i]

MutantList.remove('HO')
MutantList.remove('CAF20')

AllSample = pd.merge(locals()['mut_f_HO'],locals()['mut_f_CAF20'],on = 'gene', how = 'outer')

for i in MutantList:
    AllSample = pd.merge(AllSample,locals()[('mut_f_' + i)],on = 'gene', how = 'outer')
AllSample.to_csv((path + 'TotalMutant-RibosomeDensity.txt'), sep = '\t', na_rep = 'NA')
AllSample

,gene,HO,CAF20,EAP1,eIF4E,eIF4G1,PAB1,eIF1,eIF1A,eIF4A,eIF4B
0,YGR192C,1.044853,0.830954,0.913147,0.771685,0.541474,0.897885,0.905258,0.793557,0.985544,0.880244
1,YLR044C,1.165907,1.565413,1.930788,1.132771,1.025065,1.524017,1.376604,1.113527,1.498104,1.262894
2,YHR174W,1.380856,1.630936,1.799037,1.606966,0.929578,1.319555,2.195932,1.807041,1.609496,1.859192
3,YKL060C,1.216527,1.812802,1.668074,2.086814,1.656685,1.267185,1.416241,1.675920,1.468450,2.038270
4,YOL086C,1.406400,1.581750,1.848760,1.702193,1.071226,1.294545,1.574576,1.387739,1.361455,1.236187
...,...,...,...,...,...,...,...,...,...,...,...
6863,tD(GUC)G2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.363076
6864,tG(GCC)C,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,196.624592
6865,Q0032,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.044614
6866,tA(AGC)K1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,56.178455


In [4]:
#### spearman correlation between replications and strains

RepAndStrain_corr = pd.read_table('/Dell/Dell15/zhanggy/project/4-Ribo-seq-data/tmp/Ribo-seq_data_final/3-RibosomeDensity/RNA_Abundance/TotalMutant-RNA-Density-correlation.txt')
RepAndStrain_corr = RepAndStrain_corr.reset_index()
SampleList = ['HO.1','HO.2','CAF20.1','CAF20.2','EAP1.1','EAP1.2','eIF4E.1','eIF4E.2',\
              'eIF4G1.1','eIF4G1.2','PAB1.1','PAB1.2','eIF1.1','eIF1.2','eIF1A.1','eIF1A.2',\
                'eIF4A.1','eIF4A.2','eIF4B.1','eIF4B.2']

for i in SampleList:
    locals()[i[:-2]+'_'+i[-1]+'_corr'] = RepAndStrain_corr[['index',i]]
    locals()[i[:-2]+'_'+i[-1]+'_corr'].columns = ['index','rho']
    locals()[i[:-2]+'_'+i[-1]+'_corr']['sample'] = i
    locals()[i[:-2]+'_'+i[-1]+'_corr']['group'] = ''

    for j in range(len(locals()[i[:-2]+'_'+i[-1]+'_corr']['index'])):
        if locals()[i[:-2]+'_'+i[-1]+'_corr']['index'][j][:-2] == i[:-2]:
            locals()[i[:-2]+'_'+i[-1]+'_corr']['group'][j] = 'Rep'
        else:
            locals()[i[:-2]+'_'+i[-1]+'_corr']['group'][j] = 'Strain'

RepAndStrain_corr = pd.concat([HO_1_corr,HO_2_corr,CAF20_1_corr,CAF20_2_corr,EAP1_1_corr,EAP1_2_corr,eIF4E_1_corr,eIF4E_2_corr,\
                               eIF4G1_1_corr,eIF4G1_2_corr,PAB1_1_corr,PAB1_2_corr,eIF1_1_corr,eIF1_2_corr,eIF1A_1_corr,eIF1A_2_corr,\
                                eIF4A_1_corr,eIF4A_2_corr,eIF4B_1_corr,eIF4B_2_corr])

RepAndStrain_corr = RepAndStrain_corr[RepAndStrain_corr['rho'] != 1].reset_index(drop = True)
RepAndStrain_corr.to_csv('/Dell/Dell15/zhanggy/project/4-Ribo-seq-data/tmp/Ribo-seq_data_final/3-RibosomeDensity/RNA_Abundance/TotalMutant-RNA-Density-corr.txt', sep = '\t')
RepAndStrain_corr

/tmp/ipykernel_25597/2148960494.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  locals()[i[:-2]+'_'+i[-1]+'_corr']['sample'] = i
/tmp/ipykernel_25597/2148960494.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  locals()[i[:-2]+'_'+i[-1]+'_corr']['group'] = ''
/tmp/ipykernel_25597/2148960494.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#retur

,index,rho,sample,group
0,HO.2,0.989021,HO.1,Rep
1,CAF20.1,0.966925,HO.1,Strain
2,CAF20.2,0.961333,HO.1,Strain
3,EAP1.1,0.951856,HO.1,Strain
4,EAP1.2,0.924846,HO.1,Strain
...,...,...,...,...
375,eIF1A.1,0.962687,eIF4B.2,Strain
376,eIF1A.2,0.970510,eIF4B.2,Strain
377,eIF4A.1,0.967948,eIF4B.2,Strain
378,eIF4A.2,0.968872,eIF4B.2,Strain


 ### b. Ribosome density and Ribosome density foldchange

In [311]:
### Calculate Ribosome density

sample = 'CAF20'
#rep = '1'
name = sample# + '-' + rep
path = '/Dell/Dell15/zhanggy/project/4-Ribo-seq-data/tmp/Ribo-seq_data_final/3-RibosomeDensity/' ## The file path where -RNA-Abundance.txt and -RPF-Abundance.txt are located


mRNA_abundance = pd.read_table((path + 'RNA_Abundance/' + sample + '/' + name + '-RNA-Abundance.txt'), usecols = ['gene','RNA_abundance'])
Ribo_abundance = pd.read_table((path + 'Ribo_Abundance/' + sample + '/' + name + '-Ribo-Abundance.txt'), usecols = ['gene','Ribo_abundance'])

RibosomeDensity = pd.merge(mRNA_abundance,Ribo_abundance,on = 'gene')
RibosomeDensity['Ribosome_Density'] = RibosomeDensity['Ribo_abundance'] / RibosomeDensity['RNA_abundance']
RibosomeDensity = RibosomeDensity[['gene','RNA_abundance','Ribo_abundance','Ribosome_Density']]
RibosomeDensity.columns = ['gene','RNA_abundance','RPF_abundance','Ribosome_Density']
RibosomeDensity = RibosomeDensity[~RibosomeDensity.isin([np.nan, np.inf, -np.inf]).any(1)].reset_index().drop(['index'], axis = 1)
RibosomeDensity['sample'] = name
RibosomeDensity.to_csv((path + 'Ribosome_Density/' + sample + '/' + name + '-RibosomeDensity.txt'), sep = '\t')

In [321]:
### Calculate Ribosome density foldchange

sample = 'PAB1'
#rep = '1'
name = sample# + '-' + rep
path = ('/Dell/Dell15/zhanggy/project/4-Ribo-seq-data/tmp/Ribo-seq_data_final/3-RibosomeDensity/Ribosome_Density/' + sample + '/') ## The file path where -RibosomeDensity.txt are located

WT_RibosomeDensity = pd.read_table('/Dell/Dell15/zhanggy/project/4-Ribo-seq-data/tmp/Ribo-seq_data_final/3-RibosomeDensity/Ribosome_Density/HO/HO-RibosomeDensity.txt',\
                                    usecols = ['gene','RNA_abundance','RPF_abundance','Ribosome_Density']) ## Pathways for wild-type Ribosome Density
WT_RibosomeDensity.columns = ['gene','WT_RNA_abundance','WT_RPF_abundance','WT_Ribosome_Density']
RibosomeDensity = pd.read_table((path + name + '-RibosomeDensity.txt'), usecols = ['gene','RNA_abundance','RPF_abundance','Ribosome_Density'])

RibosomeDensity_FoldChange = pd.merge(RibosomeDensity,WT_RibosomeDensity,on = 'gene')
RibosomeDensity_FoldChange['RNA_foldchange'] = RibosomeDensity_FoldChange['RNA_abundance'] / RibosomeDensity_FoldChange['WT_RNA_abundance']
RibosomeDensity_FoldChange['RPF_foldchange'] = RibosomeDensity_FoldChange['RPF_abundance'] / RibosomeDensity_FoldChange['WT_RPF_abundance']
RibosomeDensity_FoldChange['RibosomeDensity_foldchange'] = RibosomeDensity_FoldChange['Ribosome_Density'] / RibosomeDensity_FoldChange['WT_Ribosome_Density']
RibosomeDensity_FoldChange = RibosomeDensity_FoldChange[~RibosomeDensity_FoldChange.isin([np.nan, np.inf, -np.inf]).any(1)].reset_index().drop(['index'], axis = 1)
RibosomeDensity_FoldChange.to_csv((path + name + '-RNA_RPF_RibosomeDensity-FoldChange.txt'), sep = '\t')

In [22]:
### Ribosome Density foldchange bin
path = '/Dell/Dell15/zhanggy/project/4-Ribo-seq-data/tmp/Ribo-seq_data_final/3-RibosomeDensity/RNA_Abundance/'

MutantList = ['CAF20','EAP1','eIF4E','eIF4G1','PAB1','eIF1','eIF1A','eIF4A','eIF4B']             

for i in MutantList:
    locals()[('RDfold_' + i)] = pd.read_table('/Dell/Dell15/zhanggy/project/4-Ribo-seq-data/tmp/Ribo-seq_data_final/3-RibosomeDensity/Ribosome_Density/' + i + '/' + i + '-RNA_RPF_RibosomeDensity-FoldChange.txt', usecols = ['gene','RNA_foldchange','RPF_foldchange','RibosomeDensity_foldchange'])
    locals()[('RDfold_' + i)]['sample'] = i
    locals()[('RDfold_' + i)] = locals()[('RDfold_' + i)].sort_values('RibosomeDensity_foldchange', ascending=False).reset_index(drop = True)
    total_rows = len(locals()[('RDfold_' + i)])
    top_30_percent_rows = int(total_rows * 0.3)
    bottom_30_percent_rows = int(total_rows * 0.3)
    locals()[('RDfold_' + i)]['group'] = ''
    locals()[('RDfold_' + i)].loc[:top_30_percent_rows, 'group'] = 'top'
    locals()[('RDfold_' + i)].loc[top_30_percent_rows+1:total_rows-bottom_30_percent_rows-2, 'group'] = 'median'
    locals()[('RDfold_' + i)].loc[total_rows-bottom_30_percent_rows-1:, 'group'] = 'bottom'
    locals()[('RDfold_' + i)].to_csv(('/Dell/Dell15/zhanggy/project/4-Ribo-seq-data/tmp/Ribo-seq_data_final/3-RibosomeDensity/Ribosome_Density/' + i + '/' + i + '-RNA_RPF_RibosomeDensity-FoldChange-bin.txt'),sep = '\t')


AllSample = pd.concat([locals()[('RDfold_CAF20')],locals()[('RDfold_EAP1')],locals()[('RDfold_eIF4E')],locals()[('RDfold_eIF4G1')],locals()[('RDfold_PAB1')],
                       locals()[('RDfold_eIF1')],locals()[('RDfold_eIF1A')],locals()[('RDfold_eIF4A')],locals()[('RDfold_eIF4B')]]).reset_index().drop(['index'], axis = 1)
AllSample.to_csv('/Dell/Dell15/zhanggy/project/4-Ribo-seq-data/tmp/Ribo-seq_data_final/3-RibosomeDensity/Ribosome_Density/TotalMutant-RNAandRPF-FoldChange-rbind-RDfold-bin.txt', sep = '\t')
AllSample

,gene,RNA_foldchange,RPF_foldchange,RibosomeDensity_foldchange,sample,group
0,tK(CUU)M,0.001283,5.692158,4435.138390,CAF20,top
1,tN(GUU)K,0.014053,1.956033,139.184916,CAF20,top
2,tS(AGA)D1,0.009734,1.282268,131.730625,CAF20,top
3,tG(GCC)D2,0.079949,1.946053,24.341271,CAF20,top
4,tR(CCU)J,0.185595,2.816628,15.176189,CAF20,top
...,...,...,...,...,...,...
53948,YLR154W-A,27.528981,0.295254,0.010725,eIF4B,bottom
53949,YHL037C,1.872720,0.018453,0.009854,eIF4B,bottom
53950,tL(CAA)A,16.355086,0.127677,0.007807,eIF4B,bottom
53951,snR190,0.915552,0.005464,0.005968,eIF4B,bottom


In [31]:
### uORF
Foldchange_File = pd.read_table('/Dell/Dell15/zhanggy/project/4-Ribo-seq-data/tmp/Ribo-seq_data_final/3-RibosomeDensity/Ribosome_Density/TotalMutant-foldchange-rbind.txt',usecols = ['gene','RNA_foldchange','RPF_foldchange','sample'])
Ingolia_uORF_Gene = pd.read_table('/Dell/Dell15/zhanggy/project/4-Ribo-seq-data/tmp/Ribo-seq_data/5-InitiationFactor/1-TripletPeriodicity/uORF_Gene_FristPos_2009-ingolia.txt')
Ingolia_uORF_GeneList = Ingolia_uORF_Gene['Gene'].unique()
Foldchange_File['uORF'] = ''

for i in range(len(Foldchange_File['uORF'])):
    if Foldchange_File['gene'][i] in Ingolia_uORF_GeneList:
        Foldchange_File['uORF'][i] = 'Yes'
    else:
        Foldchange_File['uORF'][i] = 'No'
Foldchange_File.to_csv('/Dell/Dell15/zhanggy/project/4-Ribo-seq-data/tmp/Ribo-seq_data_final/3-RibosomeDensity/Ribosome_Density/TotalMutant-RNAandRPF-FoldChange-rbind-uORF.txt', sep = '\t')
Foldchange_File

/tmp/ipykernel_172253/2560827747.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Foldchange_File['uORF'][i] = 'No'
/tmp/ipykernel_172253/2560827747.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Foldchange_File['uORF'][i] = 'Yes'


,gene,sample,RNA_foldchange,RPF_foldchange,uORF
0,YGR192C,HO,1.000000,1.000000,No
1,YLR044C,HO,1.000000,1.000000,No
2,YHR174W,HO,1.000000,1.000000,No
3,YKL060C,HO,1.000000,1.000000,No
4,YOL086C,HO,1.000000,1.000000,No
...,...,...,...,...,...
60052,snR60,eIF4B,0.124848,0.590508,No
60053,YCL018W,eIF4B,0.374544,1.181016,No
60054,YMR323W,eIF4B,0.374544,0.393672,No
60055,tC(GCA)B,eIF4B,0.041616,3.149375,No


In [24]:
Foldchange_File = pd.read_table('/Dell/Dell15/zhanggy/project/4-Ribo-seq-data/tmp/Ribo-seq_data_final/3-RibosomeDensity/Ribosome_Density/TotalMutant-RNAandRPF-FoldChange-rbind-uORF.txt')
Foldchange_File[(Foldchange_File['sample'] == 'PAB1')&(Foldchange_File['uORF'] == 'No')]

,Unnamed: 0,gene,sample,RNA_foldchange,RPF_foldchange,uORF
30153,30153,YGR192C,PAB1,1.084789,0.932203,No
30154,30154,YKL060C,PAB1,1.089743,1.135122,No
30155,30155,YLR044C,PAB1,0.906476,1.184901,No
30156,30156,YHR174W,PAB1,0.764134,0.730212,No
30157,30157,YOL086C,PAB1,0.953922,0.878054,No
...,...,...,...,...,...,...
36181,36181,tD(GUC)N,PAB1,1.192252,0.488894,No
36182,36182,YAR010C,PAB1,0.397417,0.570376,No
36183,36183,YBR182C-A,PAB1,0.397417,0.570376,No
36184,36184,YJR005C-A,PAB1,0.119225,0.570376,No


## 3. UMAP and KEGG pathway

 ### a. Differentialy Expressed Gene and KEGG pathway

In [12]:
### Merge the RPF Count (All reads length) into a single file (matrix) for differential expression analysis.

SeqType = 'RNA'
path1 = '/Dell/Dell15/zhanggy/project/4-Ribo-seq-data/tmp/Ribo-seq_data_final/3-RibosomeDensity/' + SeqType + '_Abundance/'
path2 = '/Dell/Dell15/zhanggy/project/4-Ribo-seq-data/tmp/Ribo-seq_data_final/4-DiffExpressGene/ColData_and_CountDataframe/' + SeqType + '/'
sample = 'PAB1'

HO_1_RPF_RNA_Count = pd.read_table((path1 + 'HO' + '/' + 'HO' + '-1_ReadCount.txt'), usecols = ['gene','0'])
HO_1_RPF_RNA_Count.columns = ['gene','HO_1']
HO_2_RPF_RNA_Count = pd.read_table((path1 + 'HO' + '/' + 'HO' + '-2_ReadCount.txt'), usecols = ['gene','0'])
HO_2_RPF_RNA_Count.columns = ['gene','HO_2']

Mutant_1_RPF_RNA_Count = pd.read_table((path1 + sample + '/' + sample + '-1_ReadCount.txt'), usecols = ['gene','0'])
Mutant_1_RPF_RNA_Count.columns = ['gene',(sample + '_1')]
Mutant_2_RPF_RNA_Count = pd.read_table((path1 + sample + '/' + sample + '-2_ReadCount.txt'), usecols = ['gene','0'])
Mutant_2_RPF_RNA_Count.columns = ['gene',(sample + '_2')]

HO_RPF_RNA_Count = pd.merge(HO_1_RPF_RNA_Count,HO_2_RPF_RNA_Count,on = 'gene')
Mutant_RPF_RNA_Count = pd.merge(Mutant_1_RPF_RNA_Count,Mutant_2_RPF_RNA_Count,on = 'gene')

RPF_RNA_Count = pd.merge(HO_RPF_RNA_Count,Mutant_RPF_RNA_Count,on = 'gene')
RPF_RNA_Count.to_csv((path2 + sample + '_CountData.txt'), sep = '\t')
RPF_RNA_Count

,gene,HO_1,HO_2,PAB1_1,PAB1_2
0,YLR044C,55981,55265,36298,48283
1,YGR192C,53449,62778,47944,57807
2,YHR174W,51074,55180,29990,38110
3,YKL060C,49285,55322,40700,54913
4,YOL086C,35230,37782,26127,32290
...,...,...,...,...,...
6356,YJL089W,1,4,6,9
6357,YKL102C,1,1,2,3
6358,YLR154W-F,1,2,2,3
6359,YLR255C,1,6,2,4


In [8]:
path = '/Dell/Dell15/zhanggy/project/4-Ribo-seq-data/tmp/Ribo-seq_data_final/4-DiffExpressGene/'

MutantList = ['CAF20','EAP1','eIF4E','eIF4G1','PAB1','eIF1','eIF1A','eIF4A','eIF4B']

for i in MutantList:
    locals()[(i + '_Diff')] = pd.read_table((path + 'ColData_and_CountDataframe/Ribo/' + i + '_DEseq2_TypeList.txt'))
    locals()[(i + '_Diff')] = locals()[(i + '_Diff')][locals()[(i + '_Diff')]['type'] == 'Significant'].reset_index(drop = True)
    locals()[(i + '_Diff')]['sample'] = i
    
    OtherStrain_GeneList = []
    MutantList_copy = MutantList.copy()

    if i in MutantList_copy:
        MutantList_copy.remove(i)
        print(f"Element '{i}' removed from the list.")
    else:
        print(f"Element '{i}' not found in the list.")
    
    for j in MutantList_copy:
        locals()[(j + '_Subset')] = pd.read_table((path + 'ColData_and_CountDataframe/Ribo/' + j + '_DEseq2_TypeList.txt'))
        locals()[(j + '_Subset')] = locals()[(j + '_Subset')][locals()[(j + '_Subset')]['type'] == 'Significant']
        OtherStrain_GeneList.extend(list(locals()[(j + '_Subset')]['row'].unique()))

    locals()[(i + '_Diff')]['Unique'] = ''
    for n in range(len(locals()[(i + '_Diff')]['Unique'])):
        gene = locals()[(i + '_Diff')]['row'][n]
        if gene in OtherStrain_GeneList:
            locals()[(i + '_Diff')]['Unique'][n] = 'No'
        elif gene not in OtherStrain_GeneList:
            locals()[(i + '_Diff')]['Unique'][n] = 'Yes'
    
    locals()[(i + '_Diff')].to_csv((path + 'ColData_and_CountDataframe/Ribo/' + i + '_DEseq2_TypeList_unique.txt'), sep = '\t')

DiffExpressGene = pd.concat([CAF20_Diff,EAP1_Diff,eIF4E_Diff,eIF4G1_Diff,\
                             PAB1_Diff,eIF1_Diff,eIF1A_Diff,eIF4A_Diff,eIF4B_Diff]).reset_index(drop = True)
DiffExpressGene.to_csv((path + 'ColData_and_CountDataframe/Ribo/' + 'AllSample_DEseq2_TypeList_Unique.txt'), sep = '\t')
DiffExpressGene

Element 'CAF20' removed from the list.
Element 'EAP1' removed from the list.
Element 'eIF4E' removed from the list.
Element 'eIF4G1' removed from the list.
Element 'PAB1' removed from the list.
Element 'eIF1' removed from the list.
Element 'eIF1A' removed from the list.
Element 'eIF4A' removed from the list.
Element 'eIF4B' removed from the list.


,row,group,type,sample,Unique
0,YLR044C,Up,Significant,CAF20,No
1,YKL060C,Up,Significant,CAF20,No
2,RDN37-1,Down,Significant,CAF20,No
3,YKL056C,Down,Significant,CAF20,No
4,RDN25-1,Down,Significant,CAF20,No
...,...,...,...,...,...
2408,YOL140W,Up,Significant,eIF4B,Yes
2409,tK(CUU)M,Up,Significant,eIF4B,No
2410,YBR013C,Up,Significant,eIF4B,No
2411,snR19,Up,Significant,eIF4B,No


In [15]:
### KEGG pathway

path = '/Dell/Dell15/zhanggy/project/4-Ribo-seq-data/tmp/Ribo-seq_data_final/4-DiffExpressGene/KEGG_pathway/Ribo/'
MutantList = ['CAF20','EAP1','eIF4E','eIF4G1','PAB1','eIF1','eIF1A','eIF4A','eIF4B']

for sample in MutantList:
    locals()[sample + '_KEGG'] = pd.read_table((path + sample + '_KEGG_Down.txt'), usecols = ['Description','qvalue'])
    locals()[sample + '_KEGG'].columns = ['Term',sample]
    locals()[sample + '_KEGG'] = locals()[sample + '_KEGG'][locals()[sample + '_KEGG'][sample] <= 0.05]
    for i in range(len(locals()[sample + '_KEGG']['Term'])):
        if len(locals()[sample + '_KEGG']['Term'][i].split(' - ')) == 2:
            locals()[sample + '_KEGG']['Term'][i] = locals()[sample + '_KEGG']['Term'][i].split(' - ')[0]
        if len(locals()[sample + '_KEGG']['Term'][i].split(' - ')) > 2:
            locals()[sample + '_KEGG']['Term'][i] = (locals()[sample + '_KEGG']['Term'][i].split(' - ')[0] + '-' + locals()[sample + '_KEGG']['Term'][i].split(' - ')[1])

KEGG_merge = pd.merge(locals()['CAF20_KEGG'],locals()['EAP1_KEGG'],on = ['Term'], how = 'outer')

MutantList.remove('CAF20')
MutantList.remove('EAP1')

for sample in MutantList:
    KEGG_merge = pd.merge(KEGG_merge,locals()[sample + '_KEGG'],on = ['Term'], how = 'outer')

#KEGG_merge['eIF1'] = 1
KEGG_merge.set_index('Term').to_csv((path + 'AllSample_KEGG_FDR_Down_0.05.txt'), sep = '\t', na_rep = 1)
KEGG_merge

/tmp/ipykernel_191002/1146808175.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  locals()[sample + '_KEGG']['Term'][i] = locals()[sample + '_KEGG']['Term'][i].split(' - ')[0]
/tmp/ipykernel_191002/1146808175.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  locals()[sample + '_KEGG']['Term'][i] = (locals()[sample + '_KEGG']['Term'][i].split(' - ')[0] + '-' + locals()[sample + '_KEGG']['Term'][i].split(' - ')[1])


,CAF20,Term,EAP1,eIF4E,eIF4G1,PAB1,eIF1,eIF1A,eIF4A,eIF4B
0,NaN,Nucleotide metabolism,0.001326,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,Ribosome,0.001740,NaN,3.318967e-06,NaN,NaN,NaN,NaN,NaN
2,NaN,Biosynthesis of secondary metabolites,NaN,0.003810,4.741484e-07,NaN,NaN,2.027062e-06,0.015612,0.000023
3,NaN,Biosynthesis of cofactors,NaN,0.003810,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,Glycolysis / Gluconeogenesis,NaN,0.003810,9.299674e-04,NaN,NaN,8.315315e-07,0.000367,0.000014
5,NaN,Protein processing in endoplasmic reticulum,NaN,0.003810,4.289198e-03,NaN,NaN,NaN,NaN,NaN
6,NaN,Various types of N-glycan biosynthesis,NaN,0.007854,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,Amino sugar and nucleotide sugar metabolism,NaN,0.010133,NaN,NaN,NaN,2.789296e-03,0.014415,0.003645
8,NaN,Fructose and mannose metabolism,NaN,0.012910,NaN,NaN,NaN,2.713044e-03,0.029639,0.000468
9,NaN,Biosynthesis of nucleotide sugars,NaN,0.012910,3.657908e-02,NaN,NaN,3.205388e-04,0.004643,0.000468


## 4. Features in different regions

### a. Reads fraction in each region

In [101]:
### Distinguishes whether a gene has a uORF

### 28nt RPF used in Ribo-seq
### all reads used in RNA-seq

sample = 'HO'
#rep = '1'
name = sample# + '-' + rep
SeqType = 'Ribo'
path1 = ('/Dell/Dell15/zhanggy/project/4-Ribo-seq-data/tmp/Ribo-seq_data_final/3-RibosomeDensity-Region/' + SeqType + '/' + sample + '/') ## The pathway of _GeneRegion_AllReadsPosition.txt
path2 = ('/Dell/Dell15/zhanggy/project/4-Ribo-seq-data/tmp/Ribo-seq_data_final/5-ReadsFraction-Region/' + SeqType + '/' + sample + '/') ## The pathway to output results

ReadsPosition = pd.read_table((path1 + name + '_GeneRegion_AllReadsPosition.txt'), usecols = ['ReadsStartPos','ReadsEndPos','gene','region'])
ReadsPosition['length'] = ReadsPosition['ReadsEndPos'] - ReadsPosition['ReadsStartPos'] + 1
ReadsPosition = ReadsPosition[ReadsPosition['length'] == 28].reset_index(drop = True) 

Ingolia_uORF_Gene = pd.read_table('/Dell/Dell15/zhanggy/project/4-Ribo-seq-data/tmp/Ribo-seq_data/5-InitiationFactor/1-TripletPeriodicity/uORF_Gene_FristPos_2009-ingolia.txt')
Ingolia_uORF_GeneList = Ingolia_uORF_Gene['Gene'].unique()
len(Ingolia_uORF_GeneList)

ReadsCount_Region_Gene_Length = ReadsPosition.groupby(['region','gene','length'])['ReadsStartPos'].count().reset_index()

def assign_value(ReadsCount_Region_Gene_Length):
    if ReadsCount_Region_Gene_Length['gene'] in Ingolia_uORF_GeneList:
        return 'Yes'
    else:
        return 'No'

ReadsCount_Region_Gene_Length['uORF'] = ReadsCount_Region_Gene_Length.apply(assign_value, axis=1) 

ReadsCount_Region_Gene_Length_uORF = ReadsCount_Region_Gene_Length.groupby(['region','uORF'])['ReadsStartPos'].sum().reset_index()
ReadsCount_Region_Gene_Length_uORF['sample'] = sample
#ReadsCount_Region_Gene_Length_uORF['rep'] = ('rep' + rep)
ReadsCount_Region_Gene_Length_uORF.to_csv((path2 + name + '_RegionReadsCount.txt'), sep = '\t')

uORF_Yes = ReadsCount_Region_Gene_Length_uORF[(ReadsCount_Region_Gene_Length_uORF['uORF'] == 'Yes')]
uORF_Yes['fraction'] = uORF_Yes['ReadsStartPos'] / uORF_Yes['ReadsStartPos'].sum()
uORF_Yes['Group'] = 'Yes'
uORF_No = ReadsCount_Region_Gene_Length_uORF[(ReadsCount_Region_Gene_Length_uORF['uORF'] == 'No')]
uORF_No['fraction'] = uORF_No['ReadsStartPos'] / uORF_No['ReadsStartPos'].sum()
uORF_No['Group'] = 'No'

uORF_Length = pd.concat([uORF_Yes,uORF_No]).reset_index(drop = True)
uORF_Length.to_csv((path2 + name + '_RegionReadsCount_LengthuORF_Fraction.txt'), sep = '\t')
uORF_Length

/tmp/ipykernel_191002/3260054085.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uORF_Yes['fraction'] = uORF_Yes['ReadsStartPos'] / uORF_Yes['ReadsStartPos'].sum()
/tmp/ipykernel_191002/3260054085.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uORF_Yes['Group'] = 'Yes'
/tmp/ipykernel_191002/3260054085.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http

,region,uORF,ReadsStartPos,sample,fraction,Group
0,3UTR,Yes,278,HO,0.018957,Yes
1,5UTR,Yes,157,HO,0.010706,Yes
2,CDS,Yes,14230,HO,0.970338,Yes
3,3UTR,No,9283,HO,0.022675,No
4,5UTR,No,498,HO,0.001216,No
5,CDS,No,399615,HO,0.976109,No


In [2]:
path = '/Dell/Dell15/zhanggy/project/4-Ribo-seq-data/tmp/Ribo-seq_data_final/5-ReadsFraction-Region/Ribo/'

MutantList = ['HO','CAF20','EAP1','eIF4E','eIF4G1','PAB1','eIF1','eIF1A','eIF4A','eIF4B']

for i in MutantList:
    locals()[('Reads_Fraction_' + i + '_1')] = pd.read_table((path + i + '/' + i + '-1_RegionReadsCount_LengthuORF_Fraction.txt'), usecols = ['region','uORF','sample','rep','fraction'])
    locals()[('Reads_Fraction_' + i + '_1')] = locals()[('Reads_Fraction_' + i + '_1')][locals()[('Reads_Fraction_' + i + '_1')]['uORF'] == 'No']
    locals()[('Reads_Fraction_' + i + '_2')] = pd.read_table((path + i + '/' + i + '-2_RegionReadsCount_LengthuORF_Fraction.txt'), usecols = ['region','uORF','sample','rep','fraction'])
    locals()[('Reads_Fraction_' + i + '_2')] = locals()[('Reads_Fraction_' + i + '_2')][locals()[('Reads_Fraction_' + i + '_2')]['uORF'] == 'No']


Reads_Fraction = pd.concat([Reads_Fraction_HO_1,Reads_Fraction_HO_2,Reads_Fraction_CAF20_1,Reads_Fraction_CAF20_2,Reads_Fraction_EAP1_1,Reads_Fraction_EAP1_2,\
                   Reads_Fraction_eIF4E_1,Reads_Fraction_eIF4E_2,Reads_Fraction_eIF4G1_1,Reads_Fraction_eIF4G1_2,Reads_Fraction_PAB1_1,Reads_Fraction_PAB1_2,\
                    Reads_Fraction_eIF1_1,Reads_Fraction_eIF1_2,Reads_Fraction_eIF1A_1,Reads_Fraction_eIF1A_2,Reads_Fraction_eIF4A_1,Reads_Fraction_eIF4A_2,\
                        Reads_Fraction_eIF4B_1,Reads_Fraction_eIF4B_2]).reset_index(drop = True)
Reads_Fraction.to_csv((path + 'AllSample_Reads_Fraction_28nt_NOuORF.txt'),sep = '\t')
Reads_Fraction

,region,uORF,sample,rep,fraction
0,3UTR,No,HO,rep1,0.023232
1,5UTR,No,HO,rep1,0.001145
2,CDS,No,HO,rep1,0.975623
3,3UTR,No,HO,rep2,0.022498
4,5UTR,No,HO,rep2,0.001239
5,CDS,No,HO,rep2,0.976263
6,3UTR,No,CAF20,rep1,0.024574
7,5UTR,No,CAF20,rep1,0.001612
8,CDS,No,CAF20,rep1,0.973813
9,3UTR,No,CAF20,rep2,0.021184


## 5. Identification of genes co-regulated by cooperative factors

### a. eIF4E/eIF4G1/PAB1/EAP1/CAF20 vs. eIF1/eIF1A/eIF4A/eIF4B

#### Cooperative Gene identification

In [34]:
path1 = '/Dell/Dell15/zhanggy/project/4-Ribo-seq-data/tmp/Ribo-seq_data_final/3-RibosomeDensity/Ribosome_Density/' ## The pathway of -RibosomeDensity.txt
path2 = '/Dell/Dell15/zhanggy/project/4-Ribo-seq-data/tmp/Ribo-seq_data_final/8-CooperativeGene/Five_vs_Four/' ## The pathway to output results

### Label 'eIF4E','eIF4G1','PAB1','EAP1','CAF20' as group1
group1_List = ['eIF4E','eIF4G1','PAB1','EAP1','CAF20']

for i in group1_List:
    locals()[i] = pd.read_table((path1 + i + '/' + i + '-RibosomeDensity.txt'), usecols = ['gene','Ribosome_Density'])
group1 = pd.concat([locals()['eIF4E'],locals()['eIF4G1'],locals()['PAB1'],locals()['EAP1'],locals()['CAF20']])
group1['type'] = 'group1'

### Label 'eIF1','eIF1A','eIF4A','eIF4B' as group2
group2_List = ['eIF1','eIF1A','eIF4A','eIF4B']

for i in group2_List:
    locals()[i] = pd.read_table((path1 + i + '/' + i + '-RibosomeDensity.txt'), usecols = ['gene','Ribosome_Density'])
group2 = pd.concat([locals()['eIF1'],locals()['eIF1A'],locals()['eIF4A'],locals()['eIF4B']])
group2['type'] = 'group2'

total_cluster = pd.concat([group1,group2])
gene1 = total_cluster['gene'].unique()

### Only consider genes with 5'UTR and 3'UTR annotations and 5'UTR length > 1
UTR_gene = pd.read_table('/Dell/Dell15/zhanggy/project/3-mRNA_looping/tmp/saccharomyces_cerevisiae_information/Saccharomyces_cerevisiae_UTRInformation.txt')
UTR3_gene = UTR_gene[UTR_gene['type'] == 'three_prime_utr']['gene_id'].unique()
UTR_5 = pd.read_table('/Dell/Dell15/zhanggy/project/3-mRNA_looping/tmp/APOBEC1_data/preview/1-GeneIdentification-Overview/5UTR_seq.txt')
UTR_5['length'] = UTR_5['stop'] - UTR_5['start'] + 1
UTR_5 = UTR_5[UTR_5['length'] != 1]
UTR5_gene = UTR_5['gene_id'].unique()
gene2 = list(set(UTR5_gene).intersection(set(UTR3_gene)))
gene = list(set(gene1).intersection(set(gene2)))

### Genes with significantly different Ribosome Density between group1 and group2 were calculated using t.test

gene_pval = open((path2 + 'Ttest_Cluster.txt'),'w')
gene_pval.write('gene\ttstat\tpval\n')

for i in gene:
    gene_subset = total_cluster[total_cluster['gene'] == i]
    gene_subset_group1 = gene_subset[gene_subset['type'] == 'group1']['Ribosome_Density'].tolist()
    gene_subset_group2 = gene_subset[gene_subset['type'] == 'group2']['Ribosome_Density'].tolist()
    tstat, pval = stats.ttest_ind(a = gene_subset_group1, b = gene_subset_group2, alternative = "two-sided")
    gene_pval.write(i + '\t' + str(tstat) + '\t' + str(pval) + '\n')
gene_pval.close()

/Dell/Dell15/zhanggy/bin/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3757: RuntimeWarning: Degrees of freedom <= 0 for slice
  return _methods._var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Dell/Dell15/zhanggy/bin/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:256: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [35]:
### Calculate the average Ribosome Density of genes in group1
group1_List = ['eIF4E','eIF4G1','PAB1','EAP1','CAF20']

for i in group1_List:
    locals()[i] = pd.read_table((path1 + i + '/' + i + '-RibosomeDensity.txt'), usecols = ['gene','Ribosome_Density'])
    locals()[i].columns = ['gene',i]
group1_List.remove('eIF4E')
group1_List.remove('eIF4G1')
group1 = pd.merge(locals()['eIF4E'],locals()['eIF4G1'],on = 'gene')
for i in group1_List:
    group1 = pd.merge(group1,locals()[i],on = 'gene')
group1['group1_RibosomeDensity_mean'] = (group1['eIF4E'] + group1['eIF4G1'] + group1['PAB1'] + group1['EAP1'] + group1['CAF20']) / 5

### Calculate the average Ribosome Density of genes in group2
group2_List = ['eIF1','eIF1A','eIF4A','eIF4B']

for i in group2_List:
    locals()[i] = pd.read_table((path1 + i + '/' + i + '-RibosomeDensity.txt'), usecols = ['gene','Ribosome_Density'])
    locals()[i].columns = ['gene',i]
group2_List.remove('eIF1')
group2_List.remove('eIF1A')
group2 = pd.merge(locals()['eIF1'],locals()['eIF1A'],on = 'gene')
for i in group2_List:
    group2 = pd.merge(group2,locals()[i],on = 'gene')
group2['group2_RibosomeDensity_mean'] = (group2['eIF1'] + group2['eIF1A'] + group2['eIF4A'] + group2['eIF4B']) / 4

### Measures the change in Ribosome Density of genes in group1 relative to group2
Groups = pd.merge(group1,group2,on = 'gene')
Groups['Ribosome_Density_foldchange'] = Groups['group1_RibosomeDensity_mean'] / Groups['group2_RibosomeDensity_mean']
Groups.to_csv((path2 + 'Cluster_Ribosome_Density_foldchange.txt'),sep = '\t')

Ttest_Cluster = pd.read_table((path2 + 'Ttest_Cluster.txt'),sep = '\t')
Ttest_Cluster = pd.merge(Groups, Ttest_Cluster, on = 'gene')
Ttest_Cluster = Ttest_Cluster[['gene','group1_RibosomeDensity_mean','group2_RibosomeDensity_mean','Ribosome_Density_foldchange','pval']]
Ttest_Cluster = Ttest_Cluster[~Ttest_Cluster.isin([np.nan, np.inf, -np.inf]).any(1)].reset_index(drop = True)

Ttest_Cluster['group'] = ''

for i in range(len(Ttest_Cluster['gene'])):
    if Ttest_Cluster['pval'][i] > 0.05:
        Ttest_Cluster['group'][i] = 'Not_significant'
    elif Ttest_Cluster['pval'][i] <= 0.05:
        if Ttest_Cluster['Ribosome_Density_foldchange'][i] > 1:
            Ttest_Cluster['group'][i] = 'Cluster1'
        elif Ttest_Cluster['Ribosome_Density_foldchange'][i] < 1:
            Ttest_Cluster['group'][i] = 'Cluster2'

Ttest_Cluster_selected = Ttest_Cluster[Ttest_Cluster['group'] != 'Not significant'].reset_index(drop = True)
print("Cluster1: ", str(len(Ttest_Cluster_selected[Ttest_Cluster_selected['group'] == 'Cluster1']['gene'].unique())))
print("Cluster2: ", str(len(Ttest_Cluster_selected[Ttest_Cluster_selected['group'] == 'Cluster2']['gene'].unique())))

Ttest_Cluster.to_csv((path2 + 'RibosomeDensity_foldchange_Pval_Cluster.txt'), sep = '\t')
#Ttest_Cluster
Ttest_Cluster

/tmp/ipykernel_69781/3318458651.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Ttest_Cluster['group'][i] = 'Not_significant'
/tmp/ipykernel_69781/3318458651.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Ttest_Cluster['group'][i] = 'Cluster2'
/tmp/ipykernel_69781/3318458651.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Ttest_Cluster['group'][i] = 'Cluster1'


Cluster1:  364
Cluster2:  205


,gene,group1_RibosomeDensity_mean,group2_RibosomeDensity_mean,Ribosome_Density_foldchange,pval,group
0,YLR044C,1.435611,1.312782,1.093564,0.554962,Not_significant
1,YGR192C,0.791029,0.891151,0.887649,0.271266,Not_significant
2,YKL060C,1.698312,1.649720,1.029455,0.810382,Not_significant
3,YHR174W,1.457214,1.867915,0.780129,0.083500,Not_significant
4,YLR075W,0.859427,0.908874,0.945594,0.740490,Not_significant
...,...,...,...,...,...,...
4261,YER011W,1.551796,1.800107,0.862058,0.476815,Not_significant
4262,YPR145C-A,2.088344,1.059272,1.971489,0.477519,Not_significant
4263,YJL037W,1.400486,0.434431,3.223723,0.052017,Not_significant
4264,YML058W-A,1.646590,1.439087,1.144191,0.497375,Not_significant


In [64]:
path2 = '/Dell/Dell15/zhanggy/project/4-Ribo-seq-data/tmp/Ribo-seq_data_final/8-CooperativeGene/Five_vs_Four/' ## The pathway to output results
Ttest_Cluster = pd.read_table((path2 + 'RibosomeDensity_foldchange_Pval_Cluster.txt'), usecols = ['gene','group'])
Ttest_Cluster[Ttest_Cluster['group'] == 'Cluster1']['gene'].to_csv((path2 + 'Cluster1_GeneList.txt'), sep = '\t')
Ttest_Cluster[Ttest_Cluster['group'] == 'Cluster2']['gene'].to_csv((path2 + 'Cluster2_GeneList.txt'), sep = '\t')

#### Features of genes in different cluster

In [ ]:
path2 = '/Dell/Dell15/zhanggy/project/4-Ribo-seq-data/tmp/Ribo-seq_data_final/8-CooperativeGene/Five_vs_Four/'
Ttest_Cluster = pd.read_table((path2 + 'RibosomeDensity_foldchange_Pval_Cluster.txt'), usecols = ['gene','Ribosome_Density_foldchange','pval','group'])

### Ribosome Density: Mutant and HO
Mutant_RibosomeDensity = pd.read_table((path2 + 'Cluster_Ribosome_Density_foldchange.txt'), usecols = ['gene','group1_RibosomeDensity_mean','group2_RibosomeDensity_mean'])
Mutant_Cluster1_RibosomeDensity = Mutant_RibosomeDensity[['gene','group2_RibosomeDensity_mean']]
Mutant_Cluster1_RibosomeDensity['sample'] = 'Cluster1_Factor_mean'
Mutant_Cluster1_RibosomeDensity.columns = ['gene','RibosomeDensity','sample']

Mutant_Cluster2_RibosomeDensity = Mutant_RibosomeDensity[['gene','group1_RibosomeDensity_mean']]
Mutant_Cluster2_RibosomeDensity['sample'] = 'Cluster2_Factor_mean'
Mutant_Cluster2_RibosomeDensity.columns = ['gene','RibosomeDensity','sample']

HO_RibosomeDensity = pd.read_table('/Dell/Dell15/zhanggy/project/4-Ribo-seq-data/tmp/Ribo-seq_data_final/3-RibosomeDensity/Ribosome_Density/HO/HO-RibosomeDensity.txt',\
                                   usecols = ['gene','Ribosome_Density','sample'])
HO_RibosomeDensity.columns = ['gene','RibosomeDensity','sample']

RibosomeDensity = pd.concat([HO_RibosomeDensity,Mutant_Cluster1_RibosomeDensity,Mutant_Cluster2_RibosomeDensity]).reset_index(drop = True)
RibosomeDensity = pd.merge(RibosomeDensity,Ttest_Cluster[['gene','group']], on = 'gene')
RibosomeDensity = RibosomeDensity[RibosomeDensity['group'] != 'Not_significant'].reset_index(drop = True)
RibosomeDensity.to_csv((path2 + 'Cluster_HOandMutant_RibosomeDensity.txt'), sep = '\t')

### Length and GC%

UTR_5_annotation = pd.read_table('/Dell/Dell15/zhanggy/project/4-Ribo-seq-data/tmp/information/5UTR_seq.txt', usecols = ['gene_id','seq'])
UTR_5_annotation['Region'] = "5'UTR"

CDS_annotation = pd.read_table('/Dell/Dell15/zhanggy/project/4-Ribo-seq-data/tmp/information/CDS_seq.txt', usecols = ['gene_id','seq'])
CDS_annotation['Region'] = "CDS"

UTR_3_annotation = pd.read_table('/Dell/Dell15/zhanggy/project/4-Ribo-seq-data/tmp/information/3UTR_seq.txt', usecols = ['gene_id','seq'])
UTR_3_annotation['Region'] = "3'UTR"

AllRegion_annotation = pd.concat([UTR_3_annotation,UTR_5_annotation,CDS_annotation]).reset_index(drop = True)

AllRegion_annotation['length'] = ''
AllRegion_annotation['GC'] = ''
for i in range(len(AllRegion_annotation['gene_id'])):
    AllRegion_annotation['length'][i] = len(AllRegion_annotation['seq'][i])
    AllRegion_annotation['GC'][i] = (AllRegion_annotation['seq'][i].count('G') + AllRegion_annotation['seq'][i].count('C')) / len(AllRegion_annotation['seq'][i])
AllRegion_annotation['GC'] = AllRegion_annotation['GC']*100
AllRegion_annotation = pd.merge(AllRegion_annotation,Ttest_Cluster_selected,left_on = 'gene_id',right_on = 'gene').drop(['gene_id'],axis = 1)

AllRegion_annotation[['gene','Region','length','GC','group']].to_csv((path2 + 'Cluster_Length_and_GC.txt'), sep = '\t')
AllRegion_annotation[['gene','Region','length','GC','group']]

### Kozak Score

KozakScore = pd.read_table('/Dell/Dell15/zhanggy/project/3-mRNA_looping/tmp/APOBEC1_data/preview/3-NormalizeFeature/Kozak/initiation_score_6+3.txt',usecols = ['gene','score_xnt'])
KozakScore.columns = ['gene','KozakScore']
KozakScore = pd.merge(Ttest_Cluster,KozakScore,on = 'gene')
KozakScore.to_csv((path2 + 'Cluster_KozakScore_6+3.txt'), sep = '\t')

### PolyA Length and nonA

PolyA_length_nonA = pd.read_table('/Dell/Dell15/zhanggy/project/3-mRNA_looping/tmp/Pacbio_data/PolyA_analyze/H_polyLength_nonAcontent.txt')
PolyA_length_nonA = pd.merge(Ttest_Cluster,PolyA_length_nonA,left_on = 'gene',right_on = 'gene_id').drop(['gene_id','Unnamed: 0'], axis = 1)
PolyA_length_nonA.to_csv((path2 + 'Cluster_PolyA.txt'), sep = '\t')

### transcript length

transcript_length = pd.read_table('/Dell/Dell15/zhanggy/project/4-Ribo-seq-data/tmp/information/AllRegionSequence.txt', usecols = ['gene_id','seq'])
transcript_length['transcript_length'] = transcript_length['seq'].str.len()
transcript_length = pd.merge(Ttest_Cluster,transcript_length[['gene_id','transcript_length']],left_on = 'gene_id',right_on = 'gene').drop(['gene_id'],axis = 1)
transcript_length.to_csv((path2 + 'Cluster_transcript_length.txt'), sep = '\t')

### uORF enrichment

Ttest_Cluster_selected = Ttest_Cluster[Ttest_Cluster['group'] != 'Not_significant'].reset_index()
Ttest_Cluster_selected = Ttest_Cluster_selected[['gene','group']]

Ingolia_uORF_Gene = pd.read_table('/Dell/Dell15/zhanggy/project/4-Ribo-seq-data/tmp/Ribo-seq_data/5-InitiationFactor/1-TripletPeriodicity/uORF_Gene_FristPos_2009-ingolia.txt')
Ingolia_uORF_GeneList = Ingolia_uORF_Gene['Gene'].unique()
Cluster1_GeneList = Ttest_Cluster_selected[Ttest_Cluster_selected['group'] == 'Cluster1']['gene'].unique()
Cluster2_GeneList = Ttest_Cluster_selected[Ttest_Cluster_selected['group'] == 'Cluster2']['gene'].unique()
print("Ingolia uORF: n = ", str(len(Ingolia_uORF_GeneList)))
print("Cluster1: n = ", str(len(Cluster1_GeneList)))
print("Cluster2: n = ", str(len(Cluster2_GeneList)))
Cluster1_uORF_GeneList = list(set(Ingolia_uORF_GeneList).intersection(set(Cluster1_GeneList)))
Cluster2_uORF_GeneList = list(set(Ingolia_uORF_GeneList).intersection(set(Cluster2_GeneList)))
print("Cluster1 have uORF: n = ", str(len(Cluster1_uORF_GeneList)))
print("Cluster2 have uORF: n = ", str(len(Cluster2_uORF_GeneList)))

uAUG_U3 = pd.read_table('/Dell/Dell15/zhanggy/project/4-Ribo-seq-data/tmp/Ribo-seq_data/5-InitiationFactor/1-TripletPeriodicity/uAUG_Gene-U3.txt')
uAUG_U3_AG = uAUG_U3[(uAUG_U3['U3'] == 'A')|(uAUG_U3['U3'] == 'G')]
U3_AG_GeneList = uAUG_U3_AG['gene'].unique()
Cluster1_uORF_U3_AG_GeneList = list(set(U3_AG_GeneList).intersection(set(Cluster1_GeneList)))
Cluster2_uORF_U3_AG_GeneList = list(set(U3_AG_GeneList).intersection(set(Cluster2_GeneList)))
print("Cluster1 have uORF U3=A/G: n = ", str(len(Cluster1_uORF_U3_AG_GeneList)))
print("Cluster2 have uORF U3=A/G: n = ", str(len(Cluster2_uORF_U3_AG_GeneList)))

uAUG_U3_CT = uAUG_U3[(uAUG_U3['U3'] == 'C')|(uAUG_U3['U3'] == 'T')]
U3_CT_GeneList = uAUG_U3_CT['gene'].unique()
Cluster1_uORF_U3_CT_GeneList = list(set(U3_CT_GeneList).intersection(set(Cluster1_GeneList)))
Cluster2_uORF_U3_CT_GeneList = list(set(U3_CT_GeneList).intersection(set(Cluster2_GeneList)))
print("Cluster1 have uORF U3=C/T: n = ", str(len(Cluster1_uORF_U3_CT_GeneList)))
print("Cluster2 have uORF U3=C/T: n = ", str(len(Cluster2_uORF_U3_CT_GeneList)))

In [11]:
path2 = '/Dell/Dell15/zhanggy/project/4-Ribo-seq-data/tmp/Ribo-seq_data_final/8-CooperativeGene/Five_vs_Four/'
Ttest_Cluster = pd.read_table((path2 + 'RibosomeDensity_foldchange_Pval_Cluster.txt'), usecols = ['gene','pval','group'])
Ttest_Cluster = Ttest_Cluster[Ttest_Cluster['group'] != 'Not_significant']


### 3-mers

transcript_annotation = pd.read_table('/Dell/Dell15/zhanggy/project/4-Ribo-seq-data/tmp/information/AllRegionSequence.txt', usecols = ['gene_id','seq'])
transcript_annotation = transcript_annotation[transcript_annotation['gene_id'].isin(Ttest_Cluster['gene'].unique())].reset_index(drop = True)
transcript_annotation['length'] = transcript_annotation['seq'].str.len()

kmers_dict = {}

for i in range(len(transcript_annotation['gene_id'])):
    gene = transcript_annotation['gene_id'][i]
    seq = transcript_annotation['seq'][i]
    length = transcript_annotation['length'][i]
    kmers_dict[gene] = []
    for j in range(length-2):
        kmers_dict[gene].append(seq[j:j+3])

kmers_df = pd.DataFrame.from_dict(kmers_dict, orient='index').T

kmers_count = pd.DataFrame()
for i in Ttest_Cluster['gene'].unique():
    kmers_count = kmers_count.append(kmers_df[i].value_counts())
kmers_count = kmers_count[kmers_count.columns.drop(list(kmers_count.filter(regex='N')))]
kmers_count = kmers_count.fillna(0).reset_index()

kmers_count = pd.merge(kmers_count,Ttest_Cluster[['gene','group']],left_on = 'index',right_on = 'gene').drop('index', axis = 1)
kmers_count = pd.merge(kmers_count,transcript_annotation[['gene_id','length']],left_on = 'gene',right_on = 'gene_id').drop('gene_id', axis = 1)

kmers_list = list(kmers_count.T.reset_index()['index'].unique()[:-3])
for i in kmers_list:
    kmers_count[i] = kmers_count[i] / (kmers_count['length'] - 2)
  
kmers_count_AGA = kmers_count[['gene','group','AGA']]
kmers_count_AGA.columns = ['gene','group','value']
kmers_count_AGA['Kmer'] = 'AGA'

kmers_count_AAG = kmers_count[['gene','group','AAG']]
kmers_count_AAG.columns = ['gene','group','value']
kmers_count_AAG['Kmer'] = 'AAG'

kmers_count_rbind = pd.concat([kmers_count_AGA,kmers_count_AAG])

kmers_list.remove('AGA')
kmers_list.remove('AAG')

for i in kmers_list:
    locals()['kmers_count_' + i] = kmers_count[['gene','group',i]]
    locals()['kmers_count_' + i].columns = ['gene','group','value']
    locals()['kmers_count_' + i]['Kmer'] = i
    kmers_count_rbind = pd.concat([kmers_count_rbind,locals()['kmers_count_' + i]]).reset_index(drop = True)
kmers_count_rbind.to_csv((path2 + 'Cluster_3mer.txt'), sep = '\t')
kmers_count_rbind

/tmp/ipykernel_323398/1813364056.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kmers_count_GCTGAA['Kmer'] = 'GCTGAA'
/tmp/ipykernel_323398/1813364056.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kmers_count_GAAGAA['Kmer'] = 'GAAGAA'
/tmp/ipykernel_323398/1813364056.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs

,gene,group,value,Kmer
0,YNL178W,Cluster2,0.007472,GCTGAA
1,YHL015W,Cluster2,0.001468,GCTGAA
2,YDR064W,Cluster2,0.000000,GCTGAA
3,YOR369C,Cluster2,0.001721,GCTGAA
4,YCR013C,Cluster2,0.000000,GCTGAA
...,...,...,...,...
2330619,YEL039C,Cluster1,0.000000,GCGTAG
2330620,YPR001W,Cluster1,0.000000,GCGTAG
2330621,YGR016W,Cluster2,0.000000,GCGTAG
2330622,YDR070C,Cluster1,0.000000,GCGTAG


#### p value and z score

In [12]:
path2 = '/Dell/Dell15/zhanggy/project/4-Ribo-seq-data/tmp/Ribo-seq_data_final/8-CooperativeGene/Five_vs_Four/' ## The pathway to output results

Length_and_GC = pd.read_table(path2 + 'Cluster_Length_and_GC.txt')
Length_and_GC = Length_and_GC[Length_and_GC['group'] != 'Not_significant'].reset_index(drop = True)

Length_Feature = Length_and_GC[['gene','group','length','Region']]
Length_Feature.columns = ['gene','group','value','Feature']
for i in range(len(Length_Feature['gene'])):
    Length_Feature['Feature'][i] = Length_Feature['Feature'][i] + '_Length'

GC_Feature = Length_and_GC[['gene','group','GC','Region']]
GC_Feature.columns = ['gene','group','value','Feature']
for i in range(len(GC_Feature['gene'])):
    GC_Feature['Feature'][i] = GC_Feature['Feature'][i] + '_GC'

KozakScore = pd.read_table(path2 + 'Cluster_KozakScore_6+3.txt')
KozakScore = KozakScore[KozakScore['group'] != 'Not_significant'].reset_index(drop = True)
KozakScore = KozakScore[['gene','group','KozakScore']]
KozakScore.columns = ['gene','group','value']
KozakScore['Feature'] = 'KozakScore'

PolyA_length_nonA = pd.read_table(path2 + 'Cluster_PolyA.txt')
PolyA_length_nonA = PolyA_length_nonA[PolyA_length_nonA['group'] != 'Not_significant'].reset_index(drop = True)

PolyA_length_Feature = PolyA_length_nonA[['gene','group','poly_length_average']]
PolyA_length_Feature.columns = ['gene','group','value']
PolyA_length_Feature['Feature'] = 'PolyA_length'

nonA_Feature = PolyA_length_nonA[['gene','group','nonA_freq_average']]
nonA_Feature.columns = ['gene','group','value']
nonA_Feature['Feature'] = 'PolyA_nonA'

transcript_length = pd.read_table((path2 + 'Cluster_transcript_length.txt'))
transcript_length = transcript_length[['gene','group','transcript_length']]
transcript_length.columns = ['gene','group','value']
transcript_length['Feature'] = 'transcript_length'
transcript_length = transcript_length[transcript_length['group'] != 'Not_significant'].reset_index(drop = True)

kmers_count_rbind = pd.read_table(path2 + 'Cluster_3mer.txt', usecols = ['gene','group','value','Kmer'])
kmers_count_rbind.columns = ['gene','group','value','Feature']


Feature_rbind = pd.concat([Length_Feature,GC_Feature,transcript_length,KozakScore,PolyA_length_Feature,nonA_Feature,kmers_count_rbind]).reset_index(drop = True)
Feature_rbind.to_csv((path2 + 'Feature_rbind_3kmer.txt'),sep = '\t')

### perform wilcox.test in R

/tmp/ipykernel_323398/3133680708.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Length_Feature['Feature'][i] = Length_Feature['Feature'][i] + '_Length'
/tmp/ipykernel_323398/3133680708.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  GC_Feature['Feature'][i] = GC_Feature['Feature'][i] + '_GC'
/tmp/ipykernel_323398/3133680708.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-

### b. eIF1/eIF1A vs. eIF4A/eIF4B

#### Cooperative Gene identification

In [38]:
path1 = '/Dell/Dell15/zhanggy/project/4-Ribo-seq-data/tmp/Ribo-seq_data_final/3-RibosomeDensity/Ribosome_Density/' ## The pathway of -RibosomeDensity.txt
path2 = '/Dell/Dell15/zhanggy/project/4-Ribo-seq-data/tmp/Ribo-seq_data_final/8-CooperativeGene/Two_vs_Two/' ## The pathway to output results

group1_List = ['eIF1','eIF1A']

for i in group1_List:
    locals()[i] = pd.read_table((path1 + i + '/' + i + '-RibosomeDensity.txt'), usecols = ['gene','Ribosome_Density'])
group1 = pd.concat([locals()['eIF1'],locals()['eIF1A']])
group1['type'] = 'group1'

group2_List = ['eIF4A','eIF4B']

for i in group2_List:
    locals()[i] = pd.read_table((path1 + i + '/' + i + '-RibosomeDensity.txt'), usecols = ['gene','Ribosome_Density'])
group2 = pd.concat([locals()['eIF4A'],locals()['eIF4B']])
group2['type'] = 'group2'

total_cluster = pd.concat([group1,group2])
gene1 = total_cluster['gene'].unique()

### Only consider genes with 5'UTR and 3'UTR annotations and 5'UTR length > 1
UTR_gene = pd.read_table('/Dell/Dell15/zhanggy/project/3-mRNA_looping/tmp/saccharomyces_cerevisiae_information/Saccharomyces_cerevisiae_UTRInformation.txt')
UTR3_gene = UTR_gene[UTR_gene['type'] == 'three_prime_utr']['gene_id'].unique()
UTR_5 = pd.read_table('/Dell/Dell15/zhanggy/project/3-mRNA_looping/tmp/APOBEC1_data/preview/1-GeneIdentification-Overview/5UTR_seq.txt')
UTR_5['length'] = UTR_5['stop'] - UTR_5['start'] + 1
UTR_5 = UTR_5[UTR_5['length'] != 1]
UTR5_gene = UTR_5['gene_id'].unique()
gene2 = list(set(UTR5_gene).intersection(set(UTR3_gene)))
gene = list(set(gene1).intersection(set(gene2)))

### Genes with significantly different Ribosome Density between group1 and group2 were calculated using t.test

gene_pval = open((path2 + 'Ttest_Cluster.txt'),'w')
gene_pval.write('gene\ttstat\tpval\n')

for i in gene:
    gene_subset = total_cluster[total_cluster['gene'] == i]
    gene_subset_group1 = gene_subset[gene_subset['type'] == 'group1']['Ribosome_Density'].tolist()
    gene_subset_group2 = gene_subset[gene_subset['type'] == 'group2']['Ribosome_Density'].tolist()
    tstat, pval = stats.ttest_ind(a = gene_subset_group1, b = gene_subset_group2, alternative = "two-sided")
    gene_pval.write(i + '\t' + str(tstat) + '\t' + str(pval) + '\n')
gene_pval.close()

/Dell/Dell15/zhanggy/bin/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3757: RuntimeWarning: Degrees of freedom <= 0 for slice
  return _methods._var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Dell/Dell15/zhanggy/bin/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:256: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [40]:
### Calculate the average Ribosome Density of genes in group1
group1_List = ['eIF1','eIF1A']

for i in group1_List:
    locals()[i] = pd.read_table((path1 + i + '/' + i + '-RibosomeDensity.txt'), usecols = ['gene','Ribosome_Density'])
    locals()[i].columns = ['gene',i]

group1 = pd.merge(locals()['eIF1'],locals()['eIF1A'],on = 'gene')
group1['group1_RibosomeDensity_mean'] = (group1['eIF1'] + group1['eIF1A']) / 2

### Calculate the average Ribosome Density of genes in group2
group2_List = ['eIF4A','eIF4B']

for i in group2_List:
    locals()[i] = pd.read_table((path1 + i + '/' + i + '-RibosomeDensity.txt'), usecols = ['gene','Ribosome_Density'])
    locals()[i].columns = ['gene',i]
group2 = pd.merge(locals()['eIF4A'],locals()['eIF4B'],on = 'gene')
group2['group2_RibosomeDensity_mean'] = (group2['eIF4A'] + group2['eIF4B']) / 2

### Measures the change in Ribosome Density of genes in group1 relative to group2
Groups = pd.merge(group1,group2,on = 'gene')
Groups['Ribosome_Density_foldchange'] = Groups['group1_RibosomeDensity_mean'] / Groups['group2_RibosomeDensity_mean']
Groups.to_csv((path2 + 'Cluster_Ribosome_Density_foldchange.txt'),sep = '\t')

Ttest_Cluster = pd.read_table((path2 + 'Ttest_Cluster.txt'),sep = '\t')
Ttest_Cluster = pd.merge(Groups, Ttest_Cluster, on = 'gene')
Ttest_Cluster = Ttest_Cluster[['gene','group1_RibosomeDensity_mean','group2_RibosomeDensity_mean','Ribosome_Density_foldchange','pval']]
Ttest_Cluster = Ttest_Cluster[~Ttest_Cluster.isin([np.nan, np.inf, -np.inf]).any(1)].reset_index(drop = True)

Ttest_Cluster['group'] = ''

for i in range(len(Ttest_Cluster['gene'])):
    if Ttest_Cluster['pval'][i] > 0.05:
        Ttest_Cluster['group'][i] = 'Not_significant'
    elif Ttest_Cluster['pval'][i] <= 0.05:
        if Ttest_Cluster['Ribosome_Density_foldchange'][i] > 1:
            Ttest_Cluster['group'][i] = 'Cluster1'
        elif Ttest_Cluster['Ribosome_Density_foldchange'][i] < 1:
            Ttest_Cluster['group'][i] = 'Cluster2'

Ttest_Cluster_selected = Ttest_Cluster[Ttest_Cluster['group'] != 'Not_significant'].reset_index(drop = True)
print("Cluster1: ", str(len(Ttest_Cluster_selected[Ttest_Cluster_selected['group'] == 'Cluster1']['gene'].unique())))
print("Cluster2: ", str(len(Ttest_Cluster_selected[Ttest_Cluster_selected['group'] == 'Cluster2']['gene'].unique())))

Ttest_Cluster.to_csv((path2 + 'RibosomeDensity_foldchange_Pval_Cluster.txt'), sep = '\t')
Ttest_Cluster

/tmp/ipykernel_69781/2598026937.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Ttest_Cluster['group'][i] = 'Not_significant'
/tmp/ipykernel_69781/2598026937.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Ttest_Cluster['group'][i] = 'Cluster2'
/tmp/ipykernel_69781/2598026937.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Ttest_Cluster['group'][i] = 'Cluster1'


Cluster1:  178
Cluster2:  50


,gene,group1_RibosomeDensity_mean,group2_RibosomeDensity_mean,Ribosome_Density_foldchange,pval,group
0,YGR192C,0.849407,0.932894,0.910508,0.390342,Not_significant
1,YLR044C,1.245066,1.380499,0.901896,0.522983,Not_significant
2,YKL060C,1.546081,1.753360,0.881782,0.576034,Not_significant
3,YLR075W,0.944501,0.873248,1.081595,0.818001,Not_significant
4,YHR174W,2.001487,1.734344,1.154031,0.367091,Not_significant
...,...,...,...,...,...,...
4273,YPR145C-A,0.847491,1.271053,0.666763,0.225026,Not_significant
4274,Q0275,13.277022,15.911548,0.834427,0.879634,Not_significant
4275,YDR034W-B,4.774208,3.202471,1.490789,0.692750,Not_significant
4276,Q0085,4.198422,6.230817,0.673816,0.749564,Not_significant


#### Features of genes in different cluster

In [95]:
path2 = '/Dell/Dell15/zhanggy/project/4-Ribo-seq-data/tmp/Ribo-seq_data_final/8-CooperativeGene/Two_vs_Two/' ## The pathway to output results
Ttest_Cluster = pd.read_table((path2 + 'RibosomeDensity_foldchange_Pval_Cluster.txt'), usecols = ['gene','Ribosome_Density_foldchange','pval','group'])

### Ribosome Density: Mutant and HO
Mutant_RibosomeDensity = pd.read_table((path2 + 'Cluster_Ribosome_Density_foldchange.txt'), usecols = ['gene','group1_RibosomeDensity_mean','group2_RibosomeDensity_mean'])
Mutant_Cluster1_RibosomeDensity = Mutant_RibosomeDensity[['gene','group2_RibosomeDensity_mean']]
Mutant_Cluster1_RibosomeDensity['sample'] = 'Cluster1_Factor_mean'
Mutant_Cluster1_RibosomeDensity.columns = ['gene','RibosomeDensity','sample']

Mutant_Cluster2_RibosomeDensity = Mutant_RibosomeDensity[['gene','group1_RibosomeDensity_mean']]
Mutant_Cluster2_RibosomeDensity['sample'] = 'Cluster2_Factor_mean'
Mutant_Cluster2_RibosomeDensity.columns = ['gene','RibosomeDensity','sample']

HO_RibosomeDensity = pd.read_table('/Dell/Dell15/zhanggy/project/4-Ribo-seq-data/tmp/Ribo-seq_data_final/3-RibosomeDensity/Ribosome_Density/HO/HO-RibosomeDensity.txt',\
                                   usecols = ['gene','Ribosome_Density','sample'])
HO_RibosomeDensity.columns = ['gene','RibosomeDensity','sample']

RibosomeDensity = pd.concat([HO_RibosomeDensity,Mutant_Cluster1_RibosomeDensity,Mutant_Cluster2_RibosomeDensity]).reset_index(drop = True)
RibosomeDensity = pd.merge(RibosomeDensity,Ttest_Cluster[['gene','group']], on = 'gene')
RibosomeDensity = RibosomeDensity[RibosomeDensity['group'] != 'Not_significant'].reset_index(drop = True)
RibosomeDensity.to_csv((path2 + 'Cluster_HOandMutant_RibosomeDensity.txt'), sep = '\t')

### Length and GC%

UTR_5_annotation = pd.read_table('/Dell/Dell15/zhanggy/project/4-Ribo-seq-data/tmp/information/5UTR_seq.txt', usecols = ['gene_id','seq'])
UTR_5_annotation['Region'] = "5'UTR"

CDS_annotation = pd.read_table('/Dell/Dell15/zhanggy/project/4-Ribo-seq-data/tmp/information/CDS_seq.txt', usecols = ['gene_id','seq'])
CDS_annotation['Region'] = "CDS"

UTR_3_annotation = pd.read_table('/Dell/Dell15/zhanggy/project/4-Ribo-seq-data/tmp/information/3UTR_seq.txt', usecols = ['gene_id','seq'])
UTR_3_annotation['Region'] = "3'UTR"

AllRegion_annotation = pd.concat([UTR_3_annotation,UTR_5_annotation,CDS_annotation]).reset_index(drop = True)

AllRegion_annotation['length'] = ''
AllRegion_annotation['GC'] = ''
for i in range(len(AllRegion_annotation['gene_id'])):
    AllRegion_annotation['length'][i] = len(AllRegion_annotation['seq'][i])
    AllRegion_annotation['GC'][i] = (AllRegion_annotation['seq'][i].count('G') + AllRegion_annotation['seq'][i].count('C')) / len(AllRegion_annotation['seq'][i])
AllRegion_annotation['GC'] = AllRegion_annotation['GC']*100
AllRegion_annotation = pd.merge(AllRegion_annotation,Ttest_Cluster,left_on = 'gene_id',right_on = 'gene').drop(['gene_id'],axis = 1)

AllRegion_annotation[['gene','Region','length','GC','group']].to_csv((path2 + 'Cluster_Length_and_GC.txt'), sep = '\t')
AllRegion_annotation[['gene','Region','length','GC','group']]

### Kozak Score

KozakScore = pd.read_table('/Dell/Dell15/zhanggy/project/3-mRNA_looping/tmp/APOBEC1_data/preview/3-NormalizeFeature/Kozak/initiation_score_6+3.txt',usecols = ['gene','score_xnt'])
KozakScore.columns = ['gene','KozakScore']
KozakScore = pd.merge(Ttest_Cluster,KozakScore,on = 'gene')
KozakScore.to_csv((path2 + 'Cluster_KozakScore_6+3.txt'), sep = '\t')

### PolyA Length and nonA

PolyA_length_nonA = pd.read_table('/Dell/Dell15/zhanggy/project/3-mRNA_looping/tmp/Pacbio_data/PolyA_analyze/H_polyLength_nonAcontent.txt')
PolyA_length_nonA = pd.merge(Ttest_Cluster,PolyA_length_nonA,left_on = 'gene',right_on = 'gene_id').drop(['gene_id','Unnamed: 0'], axis = 1)
PolyA_length_nonA.to_csv((path2 + 'Cluster_PolyA.txt'), sep = '\t')

### transcript length

transcript_length = pd.read_table('/Dell/Dell15/zhanggy/project/4-Ribo-seq-data/tmp/information/AllRegionSequence.txt', usecols = ['gene_id','seq'])
transcript_length['transcript_length'] = transcript_length['seq'].str.len()
transcript_length = pd.merge(Ttest_Cluster,transcript_length[['gene_id','transcript_length']],left_on = 'gene',right_on = 'gene_id').drop(['gene_id'],axis = 1)
transcript_length.to_csv((path2 + 'Cluster_transcript_length.txt'), sep = '\t')

### uORF enrichment

Ttest_Cluster_selected = Ttest_Cluster[Ttest_Cluster['group'] != 'Not_significant'].reset_index()
Ttest_Cluster_selected = Ttest_Cluster_selected[['gene','group']]

Ingolia_uORF_Gene = pd.read_table('/Dell/Dell15/zhanggy/project/4-Ribo-seq-data/tmp/Ribo-seq_data/5-InitiationFactor/1-TripletPeriodicity/uORF_Gene_FristPos_2009-ingolia.txt')
Ingolia_uORF_GeneList = Ingolia_uORF_Gene['Gene'].unique()
Cluster1_GeneList = Ttest_Cluster_selected[Ttest_Cluster_selected['group'] == 'Cluster1']['gene'].unique()
Cluster2_GeneList = Ttest_Cluster_selected[Ttest_Cluster_selected['group'] == 'Cluster2']['gene'].unique()
print("Ingolia uORF: n = ", str(len(Ingolia_uORF_GeneList)))
print("Cluster1: n = ", str(len(Cluster1_GeneList)))
print("Cluster2: n = ", str(len(Cluster2_GeneList)))
Cluster1_uORF_GeneList = list(set(Ingolia_uORF_GeneList).intersection(set(Cluster1_GeneList)))
Cluster2_uORF_GeneList = list(set(Ingolia_uORF_GeneList).intersection(set(Cluster2_GeneList)))
print("Cluster1 have uORF: n = ", str(len(Cluster1_uORF_GeneList)))
print("Cluster2 have uORF: n = ", str(len(Cluster2_uORF_GeneList)))

uAUG_U3 = pd.read_table('/Dell/Dell15/zhanggy/project/4-Ribo-seq-data/tmp/Ribo-seq_data/5-InitiationFactor/1-TripletPeriodicity/uAUG_Gene-U3.txt')
uAUG_U3_AG = uAUG_U3[(uAUG_U3['U3'] == 'A')|(uAUG_U3['U3'] == 'G')]
U3_AG_GeneList = uAUG_U3_AG['gene'].unique()
Cluster1_uORF_U3_AG_GeneList = list(set(U3_AG_GeneList).intersection(set(Cluster1_GeneList)))
Cluster2_uORF_U3_AG_GeneList = list(set(U3_AG_GeneList).intersection(set(Cluster2_GeneList)))
print("Cluster1 have uORF U3=A/G: n = ", str(len(Cluster1_uORF_U3_AG_GeneList)))
print("Cluster2 have uORF U3=A/G: n = ", str(len(Cluster2_uORF_U3_AG_GeneList)))

uAUG_U3_CT = uAUG_U3[(uAUG_U3['U3'] == 'C')|(uAUG_U3['U3'] == 'T')]
U3_CT_GeneList = uAUG_U3_CT['gene'].unique()
Cluster1_uORF_U3_CT_GeneList = list(set(U3_CT_GeneList).intersection(set(Cluster1_GeneList)))
Cluster2_uORF_U3_CT_GeneList = list(set(U3_CT_GeneList).intersection(set(Cluster2_GeneList)))
print("Cluster1 have uORF U3=C/T: n = ", str(len(Cluster1_uORF_U3_CT_GeneList)))
print("Cluster2 have uORF U3=C/T: n = ", str(len(Cluster2_uORF_U3_CT_GeneList)))

Ingolia uORF: n =  597
Cluster1: n =  178
Cluster2: n =  50
Cluster1 have uORF: n =  16
Cluster2 have uORF: n =  9
Cluster1 have uORF U3=A/G: n =  7
Cluster2 have uORF U3=A/G: n =  1
Cluster1 have uORF U3=C/T: n =  9
Cluster2 have uORF U3=C/T: n =  8


### c. eIF4G1/PAB1 vs. eIF4E/EAP1/CAF20

#### Cooperative Gene identification

In [41]:
path1 = '/Dell/Dell15/zhanggy/project/4-Ribo-seq-data/tmp/Ribo-seq_data_final/3-RibosomeDensity/Ribosome_Density/' ## The pathway of -RibosomeDensity.txt
path2 = '/Dell/Dell15/zhanggy/project/4-Ribo-seq-data/tmp/Ribo-seq_data_final/8-CooperativeGene/Two_vs_Three/' ## The pathway to output results

group1_List = ['eIF4G1','PAB1']

for i in group1_List:
    locals()[i] = pd.read_table((path1 + i + '/' + i + '-RibosomeDensity.txt'), usecols = ['gene','Ribosome_Density'])
group1 = pd.concat([locals()['eIF4G1'],locals()['PAB1']])
group1['type'] = 'group1'

group2_List = ['eIF4E','EAP1','CAF20']

for i in group2_List:
    locals()[i] = pd.read_table((path1 + i + '/' + i + '-RibosomeDensity.txt'), usecols = ['gene','Ribosome_Density'])
group2 = pd.concat([locals()['eIF4E'],locals()['EAP1'],locals()['CAF20']])
group2['type'] = 'group2'

total_cluster = pd.concat([group1,group2])
gene1 = total_cluster['gene'].unique()

### Only consider genes with 5'UTR and 3'UTR annotations and 5'UTR length > 1
UTR_gene = pd.read_table('/Dell/Dell15/zhanggy/project/3-mRNA_looping/tmp/saccharomyces_cerevisiae_information/Saccharomyces_cerevisiae_UTRInformation.txt')
UTR3_gene = UTR_gene[UTR_gene['type'] == 'three_prime_utr']['gene_id'].unique()
UTR_5 = pd.read_table('/Dell/Dell15/zhanggy/project/3-mRNA_looping/tmp/APOBEC1_data/preview/1-GeneIdentification-Overview/5UTR_seq.txt')
UTR_5['length'] = UTR_5['stop'] - UTR_5['start'] + 1
UTR_5 = UTR_5[UTR_5['length'] != 1]
UTR5_gene = UTR_5['gene_id'].unique()
gene2 = list(set(UTR5_gene).intersection(set(UTR3_gene)))
gene = list(set(gene1).intersection(set(gene2)))

### Genes with significantly different Ribosome Density between group1 and group2 were calculated using t.test

gene_pval = open((path2 + 'Ttest_Cluster.txt'),'w')
gene_pval.write('gene\ttstat\tpval\n')

for i in gene:
    gene_subset = total_cluster[total_cluster['gene'] == i]
    gene_subset_group1 = gene_subset[gene_subset['type'] == 'group1']['Ribosome_Density'].tolist()
    gene_subset_group2 = gene_subset[gene_subset['type'] == 'group2']['Ribosome_Density'].tolist()
    tstat, pval = stats.ttest_ind(a = gene_subset_group1, b = gene_subset_group2, alternative = "two-sided")
    gene_pval.write(i + '\t' + str(tstat) + '\t' + str(pval) + '\n')
gene_pval.close()

/Dell/Dell15/zhanggy/bin/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3757: RuntimeWarning: Degrees of freedom <= 0 for slice
  return _methods._var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Dell/Dell15/zhanggy/bin/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:256: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [42]:
### Calculate the average Ribosome Density of genes in group1
group1_List = ['eIF4G1','PAB1']

for i in group1_List:
    locals()[i] = pd.read_table((path1 + i + '/' + i + '-RibosomeDensity.txt'), usecols = ['gene','Ribosome_Density'])
    locals()[i].columns = ['gene',i]

group1 = pd.merge(locals()['eIF4G1'],locals()['PAB1'],on = 'gene')
group1['group1_RibosomeDensity_mean'] = (group1['eIF4G1'] + group1['PAB1']) / 2

### Calculate the average Ribosome Density of genes in group2
group2_List = ['eIF4E','EAP1','CAF20']

for i in group2_List:
    locals()[i] = pd.read_table((path1 + i + '/' + i + '-RibosomeDensity.txt'), usecols = ['gene','Ribosome_Density'])
    locals()[i].columns = ['gene',i]
group2 = pd.merge(locals()['eIF4E'],locals()['EAP1'],on = 'gene')
group2 = pd.merge(group2,locals()['CAF20'],on = 'gene')

group2['group2_RibosomeDensity_mean'] = (group2['eIF4E'] + group2['EAP1'] + group2['CAF20']) / 3

### Measures the change in Ribosome Density of genes in group1 relative to group2
Groups = pd.merge(group1,group2,on = 'gene')
Groups['Ribosome_Density_foldchange'] = Groups['group1_RibosomeDensity_mean'] / Groups['group2_RibosomeDensity_mean']
Groups.to_csv((path2 + 'Cluster_Ribosome_Density_foldchange.txt'),sep = '\t')

Ttest_Cluster = pd.read_table((path2 + 'Ttest_Cluster.txt'),sep = '\t')
Ttest_Cluster = pd.merge(Groups, Ttest_Cluster, on = 'gene')
Ttest_Cluster = Ttest_Cluster[['gene','group1_RibosomeDensity_mean','group2_RibosomeDensity_mean','Ribosome_Density_foldchange','pval']]
Ttest_Cluster = Ttest_Cluster[~Ttest_Cluster.isin([np.nan, np.inf, -np.inf]).any(1)].reset_index(drop = True)

Ttest_Cluster['group'] = ''

for i in range(len(Ttest_Cluster['gene'])):
    if Ttest_Cluster['pval'][i] > 0.05:
        Ttest_Cluster['group'][i] = 'Not_significant'
    elif Ttest_Cluster['pval'][i] <= 0.05:
        if Ttest_Cluster['Ribosome_Density_foldchange'][i] > 1:
            Ttest_Cluster['group'][i] = 'Cluster1'
        elif Ttest_Cluster['Ribosome_Density_foldchange'][i] < 1:
            Ttest_Cluster['group'][i] = 'Cluster2'

Ttest_Cluster_selected = Ttest_Cluster[Ttest_Cluster['group'] != 'Not_significant'].reset_index(drop = True)
print("Cluster1: ", str(len(Ttest_Cluster_selected[Ttest_Cluster_selected['group'] == 'Cluster1']['gene'].unique())))
print("Cluster2: ", str(len(Ttest_Cluster_selected[Ttest_Cluster_selected['group'] == 'Cluster2']['gene'].unique())))

Ttest_Cluster.to_csv((path2 + 'RibosomeDensity_foldchange_Pval_Cluster.txt'), sep = '\t')
Ttest_Cluster

/tmp/ipykernel_69781/552633228.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Ttest_Cluster['group'][i] = 'Not_significant'
/tmp/ipykernel_69781/552633228.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Ttest_Cluster['group'][i] = 'Cluster2'
/tmp/ipykernel_69781/552633228.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Ttest_Cluster['group'][i] = 'Cluster1'


Cluster1:  272
Cluster2:  97


,gene,group1_RibosomeDensity_mean,group2_RibosomeDensity_mean,Ribosome_Density_foldchange,pval,group
0,YGR192C,0.719680,0.838595,0.858197,0.466627,Not_significant
1,YHR174W,1.124567,1.678980,0.669792,0.043697,Cluster2
2,YKL060C,1.461935,1.855897,0.787725,0.164192,Not_significant
3,YLR044C,1.274541,1.542991,0.826020,0.500095,Not_significant
4,YLR075W,0.717548,0.954012,0.752137,0.250734,Not_significant
...,...,...,...,...,...,...
4309,YGR249W,2.189377,2.133946,1.025976,0.965942,Not_significant
4310,YPR145C-A,3.786854,0.956003,3.961132,0.309235,Not_significant
4311,YGL032C,4.773787,1.756722,2.717440,0.113953,Not_significant
4312,Q0085,6.240546,5.654282,1.103685,0.828873,Not_significant


#### Features of genes in different cluster

In [61]:
Ttest_Cluster = pd.read_table((path2 + 'RibosomeDensity_foldchange_Pval_Cluster.txt'), usecols = ['gene','Ribosome_Density_foldchange','pval','group'])

### Ribosome Density: Mutant and HO
Mutant_RibosomeDensity = pd.read_table((path2 + 'Cluster_Ribosome_Density_foldchange.txt'), usecols = ['gene','group1_RibosomeDensity_mean','group2_RibosomeDensity_mean'])
Mutant_Cluster1_RibosomeDensity = Mutant_RibosomeDensity[['gene','group2_RibosomeDensity_mean']]
Mutant_Cluster1_RibosomeDensity['sample'] = 'Cluster1_Factor_mean'
Mutant_Cluster1_RibosomeDensity.columns = ['gene','RibosomeDensity','sample']

Mutant_Cluster2_RibosomeDensity = Mutant_RibosomeDensity[['gene','group1_RibosomeDensity_mean']]
Mutant_Cluster2_RibosomeDensity['sample'] = 'Cluster2_Factor_mean'
Mutant_Cluster2_RibosomeDensity.columns = ['gene','RibosomeDensity','sample']

HO_RibosomeDensity = pd.read_table('/Dell/Dell15/zhanggy/project/4-Ribo-seq-data/tmp/Ribo-seq_data_final/3-RibosomeDensity/Ribosome_Density/HO/HO-RibosomeDensity.txt',\
                                   usecols = ['gene','Ribosome_Density','sample'])
HO_RibosomeDensity.columns = ['gene','RibosomeDensity','sample']

RibosomeDensity = pd.concat([HO_RibosomeDensity,Mutant_Cluster1_RibosomeDensity,Mutant_Cluster2_RibosomeDensity]).reset_index(drop = True)
RibosomeDensity = pd.merge(RibosomeDensity,Ttest_Cluster[['gene','group']], on = 'gene')
RibosomeDensity = RibosomeDensity[RibosomeDensity['group'] != 'Not_significant'].reset_index(drop = True)
RibosomeDensity.to_csv((path2 + 'Cluster_HOandMutant_RibosomeDensity.txt'), sep = '\t')

### Length and GC%

UTR_5_annotation = pd.read_table('/Dell/Dell15/zhanggy/project/4-Ribo-seq-data/tmp/information/5UTR_seq.txt', usecols = ['gene_id','seq'])
UTR_5_annotation['Region'] = "5'UTR"

CDS_annotation = pd.read_table('/Dell/Dell15/zhanggy/project/4-Ribo-seq-data/tmp/information/CDS_seq.txt', usecols = ['gene_id','seq'])
CDS_annotation['Region'] = "CDS"

UTR_3_annotation = pd.read_table('/Dell/Dell15/zhanggy/project/4-Ribo-seq-data/tmp/information/3UTR_seq.txt', usecols = ['gene_id','seq'])
UTR_3_annotation['Region'] = "3'UTR"

AllRegion_annotation = pd.concat([UTR_3_annotation,UTR_5_annotation,CDS_annotation]).reset_index(drop = True)

AllRegion_annotation['length'] = ''
AllRegion_annotation['GC'] = ''
for i in range(len(AllRegion_annotation['gene_id'])):
    AllRegion_annotation['length'][i] = len(AllRegion_annotation['seq'][i])
    AllRegion_annotation['GC'][i] = (AllRegion_annotation['seq'][i].count('G') + AllRegion_annotation['seq'][i].count('C')) / len(AllRegion_annotation['seq'][i])
AllRegion_annotation['GC'] = AllRegion_annotation['GC']*100
AllRegion_annotation = pd.merge(AllRegion_annotation,Ttest_Cluster_selected,left_on = 'gene_id',right_on = 'gene').drop(['gene_id'],axis = 1)

AllRegion_annotation[['gene','Region','length','GC','group']].to_csv((path2 + 'Cluster_Length_and_GC.txt'), sep = '\t')
AllRegion_annotation[['gene','Region','length','GC','group']]

### Kozak Score

KozakScore = pd.read_table('/Dell/Dell15/zhanggy/project/3-mRNA_looping/tmp/APOBEC1_data/preview/3-NormalizeFeature/Kozak/initiation_score_6+3.txt',usecols = ['gene','score_xnt'])
KozakScore.columns = ['gene','KozakScore']
KozakScore = pd.merge(Ttest_Cluster,KozakScore,on = 'gene')
KozakScore.to_csv((path2 + 'Cluster_KozakScore_6+3.txt'), sep = '\t')

### PolyA Length and nonA

PolyA_length_nonA = pd.read_table('/Dell/Dell15/zhanggy/project/3-mRNA_looping/tmp/Pacbio_data/PolyA_analyze/H_polyLength_nonAcontent.txt')
PolyA_length_nonA = pd.merge(Ttest_Cluster,PolyA_length_nonA,left_on = 'gene',right_on = 'gene_id').drop(['gene_id','Unnamed: 0'], axis = 1)
PolyA_length_nonA.to_csv((path2 + 'Cluster_PolyA.txt'), sep = '\t')

### uORF enrichment

Ttest_Cluster_selected = Ttest_Cluster[Ttest_Cluster['group'] != 'Not_significant'].reset_index()
Ttest_Cluster_selected = Ttest_Cluster_selected[['gene','group']]

Ingolia_uORF_Gene = pd.read_table('/Dell/Dell15/zhanggy/project/4-Ribo-seq-data/tmp/Ribo-seq_data/5-InitiationFactor/1-TripletPeriodicity/uORF_Gene_FristPos_2009-ingolia.txt')
Ingolia_uORF_GeneList = Ingolia_uORF_Gene['Gene'].unique()
Cluster1_GeneList = Ttest_Cluster_selected[Ttest_Cluster_selected['group'] == 'Cluster1']['gene'].unique()
Cluster2_GeneList = Ttest_Cluster_selected[Ttest_Cluster_selected['group'] == 'Cluster2']['gene'].unique()
print("Ingolia uORF: n = ", str(len(Ingolia_uORF_GeneList)))
print("Cluster1: n = ", str(len(Cluster1_GeneList)))
print("Cluster2: n = ", str(len(Cluster2_GeneList)))
Cluster1_uORF_GeneList = list(set(Ingolia_uORF_GeneList).intersection(set(Cluster1_GeneList)))
Cluster2_uORF_GeneList = list(set(Ingolia_uORF_GeneList).intersection(set(Cluster2_GeneList)))
print("Cluster1 have uORF: n = ", str(len(Cluster1_uORF_GeneList)))
print("Cluster2 have uORF: n = ", str(len(Cluster2_uORF_GeneList)))

uAUG_U3 = pd.read_table('/Dell/Dell15/zhanggy/project/4-Ribo-seq-data/tmp/Ribo-seq_data/5-InitiationFactor/1-TripletPeriodicity/uAUG_Gene-U3.txt')
uAUG_U3_AG = uAUG_U3[(uAUG_U3['U3'] == 'A')|(uAUG_U3['U3'] == 'G')]
U3_AG_GeneList = uAUG_U3_AG['gene'].unique()
Cluster1_uORF_U3_AG_GeneList = list(set(U3_AG_GeneList).intersection(set(Cluster1_GeneList)))
Cluster2_uORF_U3_AG_GeneList = list(set(U3_AG_GeneList).intersection(set(Cluster2_GeneList)))
print("Cluster1 have uORF U3=A/G: n = ", str(len(Cluster1_uORF_U3_AG_GeneList)))
print("Cluster2 have uORF U3=A/G: n = ", str(len(Cluster2_uORF_U3_AG_GeneList)))

uAUG_U3_CT = uAUG_U3[(uAUG_U3['U3'] == 'C')|(uAUG_U3['U3'] == 'T')]
U3_CT_GeneList = uAUG_U3_CT['gene'].unique()
Cluster1_uORF_U3_CT_GeneList = list(set(U3_CT_GeneList).intersection(set(Cluster1_GeneList)))
Cluster2_uORF_U3_CT_GeneList = list(set(U3_CT_GeneList).intersection(set(Cluster2_GeneList)))
print("Cluster1 have uORF U3=C/T: n = ", str(len(Cluster1_uORF_U3_CT_GeneList)))
print("Cluster2 have uORF U3=C/T: n = ", str(len(Cluster2_uORF_U3_CT_GeneList)))

Ingolia uORF: n =  597
Cluster1: n =  272
Cluster2: n =  97
Cluster1 have uORF: n =  35
Cluster2 have uORF: n =  8
Cluster1 have uORF U3=A/G: n =  9
Cluster2 have uORF U3=A/G: n =  3
Cluster1 have uORF U3=C/T: n =  26
Cluster2 have uORF U3=C/T: n =  5


## 6. Binding Affinity

### a. Ribosome Density foldchange & Binding Affinity

In [ ]:
path = '/Dell/Dell15/zhanggy/project/4-Ribo-seq-data/tmp/Ribo-seq_data/2-Binding_affinity/'
path2 = '/Dell/Dell15/zhanggy/project/4-Ribo-seq-data/tmp/Ribo-seq_data_final/3-RibosomeDensity/Ribosome_Density/'
path3 = '/Dell/Dell15/zhanggy/project/4-Ribo-seq-data/tmp/Ribo-seq_data_final/6-BindingAffinity/'

SampleList = ['CAF20','EAP1','eIF4E','eIF4G1','PAB1']

for i in SampleList:
    locals()['BindAffi_' + i] = pd.read_table((path + 'binding_affinity_' + i + '.txt'), usecols = ['genes','logFC','FDR'])
    locals()['BindAffi_' + i].columns = ['gene','logFC','FDR']
    locals()['BindAffi_' + i]['sample'] = i
    locals()['BindAffi_' + i]['group'] = ''

BindAffi = pd.concat([locals()['BindAffi_CAF20'],locals()['BindAffi_EAP1'],locals()['BindAffi_eIF4E'],\
    locals()['BindAffi_eIF4G1'],locals()['BindAffi_PAB1']]).reset_index(drop =  True)
BindAffi.to_csv(path3 + 'binding_affinity.txt', sep = '\t')

AllSample_RibosomeDensity_fold = pd.read_table(path2 + 'TotalMutant-foldchange-rbind.txt', usecols = ['gene','sample','RNA_foldchange','RPF_foldchange','RibosomeDensity_foldchange'])
AllSample_RibosomeDensity_fold = AllSample_RibosomeDensity_fold[AllSample_RibosomeDensity_fold['sample'].isin(SampleList)].reset_index(drop =  True)
AllSample_RibosomeDensity_fold = pd.merge(BindAffi,AllSample_RibosomeDensity_fold,on = ['gene','sample'])
AllSample_RibosomeDensity_fold.to_csv(path3 + 'TotalMutant-RNA_RPF_TE-Density-Fold-BindAffi.txt', sep = '\t')
AllSample_RibosomeDensity_fold

### b. PolyA length & PAB1 Binding Affinity

In [15]:
BindAffi = pd.read_table('/Dell/Dell15/zhanggy/project/4-Ribo-seq-data/tmp/Ribo-seq_data_final/6-BindingAffinity/binding_affinity.txt', usecols = ['gene','logFC','sample','group'])
PolyA_Length = pd.read_table('/Dell/Dell15/zhanggy/project/3-mRNA_looping/tmp/Pacbio_data/PolyA_analyze/H_polyLength_nonAcontent.txt')
PolyA_Length = pd.merge(BindAffi,PolyA_Length,left_on = 'gene',right_on = 'gene_id').drop('gene_id', axis = 1)
PolyA_Length.drop('Unnamed: 0', axis = 1).to_csv('/Dell/Dell15/zhanggy/project/4-Ribo-seq-data/tmp/Ribo-seq_data_final/6-BindingAffinity/PolyA_length_nonA_BindAffi.txt', sep = '\t')
PolyA_Length.drop('Unnamed: 0', axis = 1)

,gene,logFC,sample,group,poly_length_average,nonA_freq_average
0,YNR025C,2.379484,CAF20,Not_Significant,51.035985,0.001187
1,YNR025C,2.093916,EAP1,Not_Significant,51.035985,0.001187
2,YNR025C,0.413868,PAB1,Not_Significant,51.035985,0.001187
3,YOR183W,1.799892,CAF20,Not_Significant,41.251667,0.001611
4,YOR183W,0.809261,eIF4E,Not_Significant,41.251667,0.001611
...,...,...,...,...,...,...
5829,YCL041C,1.028989,eIF4G1,Not_Significant,36.613462,0.001857
5830,YCL041C,0.504321,PAB1,Not_Significant,36.613462,0.001857
5831,YIL047C-A,0.850737,eIF4G1,Not_Significant,47.437500,0.001423
5832,YIL047C-A,2.301891,PAB1,Not_Significant,47.437500,0.001423


## 7. PolyA

### a. calculate polyA Length and non-A frequency

In [ ]:
path = '/Dell/Dell15/zhanggy/project/3-mRNA_looping/tmp/Pacbio_data/PolyA_analyze/'
name = 'H-1'

PacbioGeneSeq = pd.read_table((path + name + '_mapping.txt'),usecols = ['strand','gene_id','cigar','seq'])
PacbioGeneSeq_1 = PacbioGeneSeq[PacbioGeneSeq['strand'] == '+'].reset_index().drop(['index'], axis = 1)
PacbioGeneSeq_1['poly'] = ''

for i in range(len(PacbioGeneSeq_1['gene_id'])):
    cigar = PacbioGeneSeq_1['cigar'][i]
    soft_clip = cigar.split('=')[-1]
    if "S" in soft_clip:
        soft_clip_len = - int(soft_clip.strip('S'))
        PacbioGeneSeq_1['poly'][i] = PacbioGeneSeq_1['seq'][i][soft_clip_len:]
    else:
        PacbioGeneSeq_1['poly'][i] = 'none'


PacbioGeneSeq_0 = PacbioGeneSeq[PacbioGeneSeq['strand'] == '-'].reset_index().drop(['index'], axis = 1)
PacbioGeneSeq_0['poly'] = ''

for i in range(len(PacbioGeneSeq_0['gene_id'])):
    cigar = PacbioGeneSeq_0['cigar'][i]
    soft_clip = cigar.split('=')[0]
    if "S" in soft_clip:
        soft_clip = soft_clip.split('S')[0]
        soft_clip_len = int(soft_clip)
        poly_seq = PacbioGeneSeq_0['seq'][i][:soft_clip_len]
        poly_seq = poly_seq[::-1].replace('A','t').replace('C','g').replace('G','c').replace('T','a').upper()
        PacbioGeneSeq_0['poly'][i] = poly_seq
    else:
        PacbioGeneSeq_0['poly'][i] = 'none'

PacbioGeneSeq = pd.concat([PacbioGeneSeq_1,PacbioGeneSeq_0]).reset_index().drop(['index'], axis = 1)
PacbioGeneSeq['poly_length'] = PacbioGeneSeq['poly'].apply(len)
PacbioGeneSeq.to_csv((path + name + '_polySeq.txt'), sep = '\t')

PacbioGeneSeq_step2 = PacbioGeneSeq[['gene_id','poly','poly_length']]

PacbioGeneSeq_step2['polyA_A_freq'] = ''
PacbioGeneSeq_step2['polyA_nonA_count'] = ''

for i in range(len(PacbioGeneSeq_step2['poly'])):
    PolyA_seq = PacbioGeneSeq_step2['poly'][i]
    PolyA_len = PacbioGeneSeq_step2['poly_length'][i]
    PacbioGeneSeq_step2['polyA_A_freq'][i] = PolyA_seq.count('A') / PolyA_len
    PacbioGeneSeq_step2['polyA_nonA_count'][i] = (PolyA_seq.count('G') + PolyA_seq.count('C') + PolyA_seq.count('T'))

PacbioGeneSeq_cutoff = PacbioGeneSeq_step2[(PacbioGeneSeq_step2['poly_length'] >= 15) & (PacbioGeneSeq['poly_length'] <= 500) & \
    (PacbioGeneSeq_step2['polyA_A_freq'] >= 0.5) & (PacbioGeneSeq_step2['polyA_nonA_count'] <= 20)].reset_index().drop(['index'], axis = 1)

num_df = PacbioGeneSeq_cutoff['gene_id'].value_counts()
num_df = num_df.reset_index()
num_df.columns = ['gene','num']
num_df = num_df[num_df['num'] >= 10].reset_index().drop(['index'], axis = 1)
gene = num_df['gene'].unique()

PacbioGeneSeq_cutoff = PacbioGeneSeq_cutoff[PacbioGeneSeq_cutoff['gene_id'].isin(gene)].reset_index().drop(['index'], axis = 1)
PacbioGeneSeq_cutoff['nonA_freq'] = 1 - PacbioGeneSeq_cutoff['polyA_A_freq']
PacbioGeneSeq_cutoff.to_csv((path + name + '_polySeq_before_average.txt'), sep = '\t')

length_df = PacbioGeneSeq_cutoff[['gene_id','poly_length']].groupby(['gene_id']).mean()
nonA_df = PacbioGeneSeq_cutoff[['gene_id','nonA_freq']].groupby(['gene_id']).mean()

df_length_nonA = pd.merge(length_df,nonA_df,on = 'gene_id')
df_length_nonA.to_csv((path + name + '_polyLength_nonAcontent.txt'), sep = '\t')

In [ ]:
sample1 = pd.read_table('/Dell/Dell15/zhanggy/project/3-mRNA_looping/tmp/Pacbio_data/PolyA_analyze/H-1_polyLength_nonAcontent.txt')
sample2 = pd.read_table('/Dell/Dell15/zhanggy/project/3-mRNA_looping/tmp/Pacbio_data/PolyA_analyze/H-2_polyLength_nonAcontent.txt')
sample3 = pd.read_table('/Dell/Dell15/zhanggy/project/3-mRNA_looping/tmp/Pacbio_data/PolyA_analyze/H-3_polyLength_nonAcontent.txt')

total_1 = pd.merge(sample1,sample2,on = 'gene_id')
total = pd.merge(total_1,sample3,on = 'gene_id')

total['poly_length_average'] = (total['poly_length_x'] + total['poly_length_y'] + total['poly_length']) / 3
total['nonA_freq_average'] = (total['nonA_freq_x'] + total['nonA_freq_y'] + total['nonA_freq']) / 3
total = total[['gene_id','poly_length_average','nonA_freq_average']]
total.to_csv('/Dell/Dell15/zhanggy/project/3-mRNA_looping/tmp/Pacbio_data/PolyA_analyze/H_polyLength_nonAcontent.txt', sep = '\t')


### b. polyA Length and Ribosome Density(fold change)

In [26]:
path2 = '/Dell/Dell15/zhanggy/project/4-Ribo-seq-data/tmp/Ribo-seq_data_final/3-RibosomeDensity/Ribosome_Density/'
path3 = '/Dell/Dell15/zhanggy/project/4-Ribo-seq-data/tmp/Ribo-seq_data_final/7-PolyA/'

polyA_length = pd.read_table('/Dell/Dell15/zhanggy/project/3-mRNA_looping/tmp/Pacbio_data/PolyA_analyze/H_polyLength_nonAcontent.txt', usecols = ['gene_id','poly_length_average'])
#polyA_length = pd.read_csv('/Dell/Dell15/zhanggy/project/4-Ribo-seq-data/tmp/Ribo-seq_data_final/7-PolyA/paper_data/NormalizedCountsMeanMedianTable1.csv', usecols = ['Systematic.Name','WT.BY.mean'])
polyA_length.columns = ['gene_id','poly_length_average']

polyA_length_sorted = polyA_length.sort_values('poly_length_average').reset_index(drop = True)
num_bins = 8
bin_size = len(polyA_length_sorted) // num_bins

bins = [i * bin_size for i in range(num_bins + 1)]
bin_labels = [f'bin{i + 1}' for i in range(num_bins)]
polyA_length_sorted['bin_label'] = pd.cut(polyA_length_sorted.index, bins=bins, labels=bin_labels, right=False)
polyA_length_sorted = polyA_length_sorted.fillna(bin_labels[-1])

# Ribosome Density
Allsample_RibosomeDensity = pd.read_table((path2 + 'TotalMutant-foldchange-rbind.txt'), usecols = ['gene','Ribosome_Density','sample'])

polyA_length_sorted_1 = pd.merge(polyA_length_sorted,Allsample_RibosomeDensity,left_on = 'gene_id',right_on = 'gene').drop('gene_id', axis = 1)
polyA_length_sorted_1.to_csv((path3 + 'PolyALength_and_RibosomeDensity.txt'), sep = '\t')

# Ribosome Density foldchange
Allsample_RibosomeDensity_fold = pd.read_table((path2 + 'TotalMutant-foldchange-rbind.txt'), usecols = ['gene','RibosomeDensity_foldchange','sample'])
Allsample_RibosomeDensity_fold = Allsample_RibosomeDensity_fold[~Allsample_RibosomeDensity_fold.isin([np.nan, np.inf, -np.inf]).any(1)].reset_index().drop(['index'], axis = 1)
polyA_length_sorted_2 = pd.merge(polyA_length_sorted,Allsample_RibosomeDensity_fold,left_on = 'gene_id',right_on = 'gene').drop(['gene_id'], axis = 1)
polyA_length_sorted_2.to_csv((path3 + 'PolyALength_and_RibosomeDensity-fold.txt'), sep = '\t')


### c. polyA Length and nonA content

In [4]:
polyA_length_nonA = pd.read_table('/Dell/Dell15/zhanggy/project/3-mRNA_looping/tmp/Pacbio_data/PolyA_analyze/H_polyLength_nonAcontent.txt')
np.mean(polyA_length_nonA['poly_length_average'])

46.08437037379161

## 8. mRNA

### a. CDS Length ~ Ribosome Density(fold change)

In [27]:
path2 = '/Dell/Dell15/zhanggy/project/4-Ribo-seq-data/tmp/Ribo-seq_data_final/3-RibosomeDensity/Ribosome_Density/'
path3 = '/Dell/Dell15/zhanggy/project/4-Ribo-seq-data/tmp/Ribo-seq_data_final/9-mRNA/'

CDS_length = pd.read_table('/Dell/Dell15/zhanggy/project/4-Ribo-seq-data/tmp/information/CDS_seq.txt', usecols = ['gene_id','seq'])
CDS_length['length'] = CDS_length['seq'].str.len()
CDS_length = CDS_length[['gene_id','length']]
CDS_length.columns = ['gene_id','length']

CDS_length_sorted = CDS_length.sort_values('length').reset_index(drop = True)
num_bins = 8
bin_size = len(CDS_length_sorted) // num_bins

bins = [i * bin_size for i in range(num_bins + 1)]
bin_labels = [f'bin{i + 1}' for i in range(num_bins)]
CDS_length_sorted['bin_label'] = pd.cut(CDS_length_sorted.index, bins=bins, labels=bin_labels, right=False)
CDS_length_sorted = CDS_length_sorted.fillna(bin_labels[-1])

# Ribosome Density
Allsample_RibosomeDensity = pd.read_table((path2 + 'TotalMutant-foldchange-rbind.txt'), usecols = ['gene','Ribosome_Density','sample'])

CDS_length_sorted_1 = pd.merge(CDS_length_sorted,Allsample_RibosomeDensity,left_on = 'gene_id',right_on = 'gene').drop('gene_id', axis = 1)
CDS_length_sorted_1.to_csv((path3 + 'CDSLength_and_RibosomeDensity.txt'), sep = '\t')

# Ribosome Density foldchange
Allsample_RibosomeDensity_fold = pd.read_table((path2 + 'TotalMutant-foldchange-rbind.txt'), usecols = ['gene','RibosomeDensity_foldchange','sample'])
Allsample_RibosomeDensity_fold = Allsample_RibosomeDensity_fold[~Allsample_RibosomeDensity_fold.isin([np.nan, np.inf, -np.inf]).any(1)].reset_index().drop(['index'], axis = 1)
CDS_length_sorted_2 = pd.merge(CDS_length_sorted,Allsample_RibosomeDensity_fold,left_on = 'gene_id',right_on = 'gene').drop(['gene_id'], axis = 1)
CDS_length_sorted_2.to_csv((path3 + 'CDSLength_and_RibosomeDensity-fold.txt'), sep = '\t')
